In [1]:
import gym
# Tensorflow Library
import tensorflow as tf
import numpy as np
from collections import deque
from time import time, strftime, localtime
import os
tf.get_logger().setLevel('ERROR')
import matplotlib.pyplot as plt

In [2]:
!mkdir test

A subdirectory or file test already exists.


In [8]:
import gym, random, tempfile
import numpy as np
from gym import wrappers
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

class DDQL:
    def __init__(self, nS, nA):
        self.nS = nS
        self.nA = nA
        self.epsilon = 1
        self.max_e = 1
        self.min_e = 0.0001
        self.epsilon_decay = 0.002
        self.gamma = 0.99
        self.learning_rate = 0.0005
        self.epochs = 1
        self.verbose = 0
        self.minibatch_size = 30
        self.memory = deque(maxlen=5000)
        self.model = self.create_model()
        self.target_model = self.create_model()

    def create_model(self):
        model = Sequential()

        # Add 2 hidden layers with 64 nodes each
        model.add(Dense(64, input_dim=self.nS, activation='relu'))
        model.add(Dense(64, activation='relu'))
        model.add(Dense(self.nA, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
        return model

    def add_memory(self, s, a, r, s_prime, done):
        self.memory.append((s, a, r, s_prime, done))

    def target_model_update(self):
        self.target_model.set_weights(self.model.get_weights())

    def selectAction(self, s):
        if np.random.rand() <= self.epsilon:
            return np.random.choice(self.nA)
        q = self.model.predict(s)
        return np.argmax(q[0])

    def replay(self):
        # Vectorized method for experience replay
        minibatch = random.sample(self.memory, self.minibatch_size)
        minibatch = np.array(minibatch)
        not_done_indices = np.where(minibatch[:, 4] == False)
        y = np.copy(minibatch[:, 2])

        # If minibatch contains any non-terminal states, use separate update rule for those states
        if len(not_done_indices[0]) > 0:
            predict_sprime = self.model.predict(np.vstack(minibatch[:, 3]))
            predict_sprime_target = self.target_model.predict(np.vstack(minibatch[:, 3]))
            
            # Non-terminal update rule
            y[not_done_indices] += np.multiply(self.gamma, \
                        predict_sprime_target[not_done_indices, \
                        np.argmax(predict_sprime[not_done_indices, :][0], axis=1)][0])

        actions = np.array(minibatch[:, 1], dtype=int)
        y_target = self.model.predict(np.vstack(minibatch[:, 0]))
        y_target[range(self.minibatch_size), actions] = y
        self.model.fit(np.vstack(minibatch[:, 0]), y_target, epochs=self.epochs, verbose=self.verbose)

    def replayIterative(self):
        # Iterative method - this performs the same function as replay() but is not vectorized 
        s_list = []
        y_state_list = []
        minibatch = random.sample(self.memory, self.minibatch_size)
        for s, a, r, s_prime, done in minibatch:
            s_list.append(s)
            y_action = r
            if not done:
                y_action = r + self.gamma * np.amax(self.model.predict(s_prime)[0])

            print(y_action)
            
            y_state = self.model.predict(s)
            y_state[0][a] = y_action
            y_state_list.append(y_state)
        self.model.fit(np.squeeze(s_list), np.squeeze(y_state_list), batch_size=self.minibatch_size, epochs=1, verbose=0)

if __name__ == "__main__":
    np.set_printoptions(precision=2)
    tdir = tempfile.mkdtemp()
    env = gym.make('LunarLander-v2')
    env = wrappers.Monitor(env, tdir, force=True, video_callable=False)

    nS = env.observation_space.shape[0]
    nA = env.action_space.n

    agent = DDQL(nS, nA)

    # Set to true to use saved model
    episodes = 10000

    # Cumulative reward
    reward_avg = deque(maxlen=100)

    for e in range(1295,episodes):
        episode_reward = 0
        s = env.reset()
        s = np.reshape(s, [1, nS])

        for time in range(1000):

            # Query next action from learner and perform action
            a = agent.selectAction(s)
            s_prime, r, done, info = env.step(a)

            # Add cumulative reward
            episode_reward += r

            # Reshape new state
            s_prime = np.reshape(s_prime, [1, nS])

            # Add experience to memory
            agent.add_memory(s, a, r, s_prime, done)

            # Set current state to new state
            s = s_prime

            #Perform experience replay if memory length is greater than minibatch length
   
            if len(agent.memory) > agent.minibatch_size:
                agent.replay()

            # If episode is done, exit loop
            if done:
                agent.target_model_update()
                break


        # epsilon decay
        if agent.epsilon > agent.min_e:
            agent.epsilon *= agent.epsilon_decay
            agent.epsilon = agent.min_e + (agent.max_e - agent.min_e)*np.exp(-agent.epsilon_decay*e)

        # Running average of past 100 episodes
        reward_avg.append(episode_reward)
        print('episode: ', e, ' score: ', '%.2f' % episode_reward, ' avg_score: ', '%.2f' % np.average(reward_avg), ' frames: ', time, ' epsilon: ', '%.2f' % agent.epsilon)
        
        # with open('trained_agent.txt', 'a') as f:
        #     f.write(str(np.average(reward_avg)) + '\n')

    env.close()


agent.model.save_weights('weights_DQN_'+str(agent.max_e)+'-'+str(agent.min_e)+'-'+str(agent.epsilon_decay)\
                         +'-'+str(agent.gamma)+'-'+str(agent.minibatch_size)+'-'+str(agent.learning_rate)+'.h5')

episode:  0  score:  -217.80  avg_score:  -217.80  frames:  129  epsilon:  1.00
episode:  1  score:  -133.61  avg_score:  -175.70  frames:  104  epsilon:  1.00
episode:  2  score:  -78.27  avg_score:  -143.22  frames:  77  epsilon:  1.00
episode:  3  score:  -100.48  avg_score:  -132.54  frames:  72  epsilon:  0.99
episode:  4  score:  -57.84  avg_score:  -117.60  frames:  67  epsilon:  0.99
episode:  5  score:  -76.59  avg_score:  -110.76  frames:  60  epsilon:  0.99
episode:  6  score:  -311.44  avg_score:  -139.43  frames:  92  epsilon:  0.99
episode:  7  score:  -92.20  avg_score:  -133.53  frames:  91  epsilon:  0.99
episode:  8  score:  -201.70  avg_score:  -141.10  frames:  78  epsilon:  0.98
episode:  9  score:  -137.61  avg_score:  -140.75  frames:  103  epsilon:  0.98
episode:  10  score:  -96.70  avg_score:  -136.75  frames:  138  epsilon:  0.98
episode:  11  score:  -99.51  avg_score:  -133.65  frames:  86  epsilon:  0.98
episode:  12  score:  -110.50  avg_score:  -131.87  

episode:  103  score:  -92.00  avg_score:  -139.96  frames:  115  epsilon:  0.81
episode:  104  score:  -94.06  avg_score:  -140.32  frames:  74  epsilon:  0.81
episode:  105  score:  -50.45  avg_score:  -140.06  frames:  71  epsilon:  0.81
episode:  106  score:  -190.11  avg_score:  -138.85  frames:  100  epsilon:  0.81
episode:  107  score:  -15.36  avg_score:  -138.08  frames:  122  epsilon:  0.81
episode:  108  score:  -149.08  avg_score:  -137.55  frames:  64  epsilon:  0.81
episode:  109  score:  -69.14  avg_score:  -136.87  frames:  78  epsilon:  0.80
episode:  110  score:  -139.24  avg_score:  -137.29  frames:  81  epsilon:  0.80
episode:  111  score:  -74.54  avg_score:  -137.04  frames:  74  epsilon:  0.80
episode:  112  score:  -47.72  avg_score:  -136.41  frames:  117  epsilon:  0.80
episode:  113  score:  -41.08  avg_score:  -135.00  frames:  71  epsilon:  0.80
episode:  114  score:  -109.89  avg_score:  -134.42  frames:  79  epsilon:  0.80
episode:  115  score:  -145.00  

episode:  206  score:  -68.83  avg_score:  -76.41  frames:  147  epsilon:  0.66
episode:  207  score:  -15.10  avg_score:  -76.41  frames:  112  epsilon:  0.66
episode:  208  score:  -215.05  avg_score:  -77.07  frames:  100  epsilon:  0.66
episode:  209  score:  8.91  avg_score:  -76.29  frames:  166  epsilon:  0.66
episode:  210  score:  -42.44  avg_score:  -75.32  frames:  122  epsilon:  0.66
episode:  211  score:  -40.93  avg_score:  -74.99  frames:  82  epsilon:  0.66
episode:  212  score:  -52.78  avg_score:  -75.04  frames:  104  epsilon:  0.65
episode:  213  score:  -50.21  avg_score:  -75.13  frames:  76  epsilon:  0.65
episode:  214  score:  -55.33  avg_score:  -74.58  frames:  124  epsilon:  0.65
episode:  215  score:  -22.70  avg_score:  -73.36  frames:  149  epsilon:  0.65
episode:  216  score:  -100.64  avg_score:  -73.92  frames:  176  epsilon:  0.65
episode:  217  score:  -63.28  avg_score:  -74.05  frames:  116  epsilon:  0.65
episode:  218  score:  -58.85  avg_score: 

episode:  309  score:  -18.93  avg_score:  -55.54  frames:  116  epsilon:  0.54
episode:  310  score:  -42.04  avg_score:  -55.54  frames:  126  epsilon:  0.54
episode:  311  score:  -16.28  avg_score:  -55.29  frames:  82  epsilon:  0.54
episode:  312  score:  -27.05  avg_score:  -55.04  frames:  999  epsilon:  0.54
episode:  313  score:  -28.12  avg_score:  -54.81  frames:  123  epsilon:  0.53
episode:  314  score:  -16.57  avg_score:  -54.43  frames:  138  epsilon:  0.53
episode:  315  score:  -63.74  avg_score:  -54.84  frames:  118  epsilon:  0.53
episode:  316  score:  -52.72  avg_score:  -54.36  frames:  126  epsilon:  0.53
episode:  317  score:  -56.12  avg_score:  -54.29  frames:  107  epsilon:  0.53
episode:  318  score:  -67.97  avg_score:  -54.38  frames:  102  epsilon:  0.53
episode:  319  score:  -41.74  avg_score:  -54.46  frames:  160  epsilon:  0.53
episode:  320  score:  -11.76  avg_score:  -52.68  frames:  140  epsilon:  0.53
episode:  321  score:  -211.66  avg_score

episode:  412  score:  131.87  avg_score:  -34.65  frames:  999  epsilon:  0.44
episode:  413  score:  54.87  avg_score:  -33.82  frames:  999  epsilon:  0.44
episode:  414  score:  14.75  avg_score:  -33.51  frames:  118  epsilon:  0.44
episode:  415  score:  6.00  avg_score:  -32.81  frames:  114  epsilon:  0.44
episode:  416  score:  -44.94  avg_score:  -32.73  frames:  159  epsilon:  0.44
episode:  417  score:  -33.45  avg_score:  -32.50  frames:  125  epsilon:  0.43
episode:  418  score:  -22.84  avg_score:  -32.05  frames:  111  epsilon:  0.43
episode:  419  score:  -22.83  avg_score:  -31.86  frames:  130  epsilon:  0.43
episode:  420  score:  -205.06  avg_score:  -33.80  frames:  166  epsilon:  0.43
episode:  421  score:  57.30  avg_score:  -31.11  frames:  999  epsilon:  0.43
episode:  422  score:  -116.87  avg_score:  -31.50  frames:  999  epsilon:  0.43
episode:  423  score:  24.81  avg_score:  -31.15  frames:  143  epsilon:  0.43
episode:  424  score:  108.86  avg_score:  -

...

episode:  932  score:  -42.09  avg_score:  134.09  frames:  128  epsilon:  0.16
episode:  933  score:  -96.61  avg_score:  130.34  frames:  135  epsilon:  0.15
episode:  934  score:  293.19  avg_score:  133.27  frames:  244  epsilon:  0.15
episode:  935  score:  262.42  avg_score:  133.24  frames:  267  epsilon:  0.15
episode:  936  score:  140.72  avg_score:  131.97  frames:  913  epsilon:  0.15
episode:  937  score:  -25.50  avg_score:  130.11  frames:  82  epsilon:  0.15
episode:  938  score:  -220.28  avg_score:  125.10  frames:  173  epsilon:  0.15
episode:  939  score:  -80.69  avg_score:  121.81  frames:  157  epsilon:  0.15
episode:  940  score:  16.64  avg_score:  119.85  frames:  88  epsilon:  0.15
episode:  941  score:  229.25  avg_score:  119.32  frames:  615  epsilon:  0.15
episode:  942  score:  -23.67  avg_score:  116.64  frames:  65  epsilon:  0.15
episode:  943  score:  -15.67  avg_score:  114.09  frames:  107  epsilon:  0.15
episode:  944  score:  -17.43  avg_score:  

episode:  1035  score:  -36.03  avg_score:  104.13  frames:  114  epsilon:  0.13
episode:  1036  score:  -16.88  avg_score:  102.55  frames:  88  epsilon:  0.13
episode:  1037  score:  268.04  avg_score:  105.49  frames:  190  epsilon:  0.13
episode:  1038  score:  -377.98  avg_score:  103.91  frames:  403  epsilon:  0.13
episode:  1039  score:  -153.08  avg_score:  103.19  frames:  132  epsilon:  0.13
episode:  1040  score:  232.08  avg_score:  105.34  frames:  238  epsilon:  0.13
episode:  1041  score:  -485.37  avg_score:  98.19  frames:  91  epsilon:  0.12
episode:  1042  score:  230.93  avg_score:  100.74  frames:  405  epsilon:  0.12
episode:  1043  score:  264.97  avg_score:  103.55  frames:  598  epsilon:  0.12
episode:  1044  score:  -50.85  avg_score:  103.21  frames:  398  epsilon:  0.12
episode:  1045  score:  257.68  avg_score:  103.75  frames:  247  epsilon:  0.12
episode:  1046  score:  228.65  avg_score:  106.55  frames:  494  epsilon:  0.12
episode:  1047  score:  259.

episode:  1137  score:  267.63  avg_score:  174.60  frames:  301  epsilon:  0.10
episode:  1138  score:  22.77  avg_score:  178.61  frames:  133  epsilon:  0.10
episode:  1139  score:  234.59  avg_score:  182.48  frames:  254  epsilon:  0.10
episode:  1140  score:  258.61  avg_score:  182.75  frames:  376  epsilon:  0.10
episode:  1141  score:  32.15  avg_score:  187.92  frames:  164  epsilon:  0.10
episode:  1142  score:  262.35  avg_score:  188.24  frames:  446  epsilon:  0.10
episode:  1143  score:  253.12  avg_score:  188.12  frames:  329  epsilon:  0.10
episode:  1144  score:  300.22  avg_score:  191.63  frames:  250  epsilon:  0.10
episode:  1145  score:  247.96  avg_score:  191.53  frames:  542  epsilon:  0.10
episode:  1146  score:  -0.00  avg_score:  189.25  frames:  144  epsilon:  0.10
episode:  1147  score:  264.74  avg_score:  189.29  frames:  325  epsilon:  0.10
episode:  1148  score:  282.35  avg_score:  189.60  frames:  679  epsilon:  0.10
episode:  1149  score:  261.22 

episode:  1239  score:  228.55  avg_score:  164.41  frames:  443  epsilon:  0.08
episode:  1240  score:  280.84  avg_score:  164.63  frames:  258  epsilon:  0.08
episode:  1241  score:  249.34  avg_score:  166.80  frames:  238  epsilon:  0.08
episode:  1242  score:  276.77  avg_score:  166.94  frames:  254  epsilon:  0.08
episode:  1243  score:  230.47  avg_score:  166.72  frames:  236  epsilon:  0.08
episode:  1244  score:  260.39  avg_score:  166.32  frames:  312  epsilon:  0.08
episode:  1245  score:  249.95  avg_score:  166.34  frames:  318  epsilon:  0.08
episode:  1246  score:  265.79  avg_score:  169.00  frames:  252  epsilon:  0.08
episode:  1247  score:  233.66  avg_score:  168.69  frames:  259  epsilon:  0.08
episode:  1248  score:  261.21  avg_score:  168.47  frames:  255  epsilon:  0.08
episode:  1249  score:  253.85  avg_score:  168.40  frames:  229  epsilon:  0.08
episode:  1250  score:  280.72  avg_score:  168.36  frames:  277  epsilon:  0.08
episode:  1251  score:  278.

KeyboardInterrupt: 

In [35]:
import gym, random, tempfile
import numpy as np
from gym import wrappers
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
agent.model.load_weights('weights_DQN_'+str(agent.max_e)+'-'+str(agent.min_e)+'-'+str(agent.epsilon_decay)\
                     +'-'+str(agent.gamma)+'-'+str(agent.minibatch_size)+'-'+str(agent.learning_rate)+'.h5')

# Set to true to use saved model
episodes = 10000
env = gym.make('LunarLander-v2')
# Cumulative reward
reward_avg = [269.47,
 243.59,
 263.63,
 42.84,
 269.43,
 26.39,
 229.25,
 263.5,
 254.84,
 -371.86,
 16.64,
 32.61,
 267.62,
 -29.69,
 -13.96,
 -409.28,
 -165.61,
 197.65,
 146.39,
 229.76,
 259.79,
 -351.97,
 230.21,
 -367.69,
 252.92,
 211.64,
 13.18,
 254.74,
 264.83,
 137.21,
 2.14,
 172.48,
 238.69,
 264.28,
 278.16,
 268.74,
 288.04,
 295.54,
 245.06,
 260.62,
 295.37,
 315.5,
 294.7,
 228.55,
 280.84,
 249.34,
 276.77,
 230.47,
 260.39,
 249.95,
 265.79,
 233.66,
 261.21,
 253.85,
 280.72,
 278.8,
 298.21,
 255.76,
 256.29,
 270.51,
 285.1,
 268.92,
 275.22,
 262.38,
 244.21,
 271.11,
 247.02,
 216.66,
 286.94,
 282.25,
 256.29,
 281.3,
 269.76,
 248.96,
 268.45,
 216.47,
 273.39,
 275.06,
 250.31,
 275.99,
 257.99,
 -130.06,
 231.62,
 281.62,
 285.23,
 277.81,
 282.65,
 281.32,
 234.27,
 270.24,
 289.71,
 -5.36,
 244.64,
 250.3,
 -313.06,
 252.85,
 274.53,
 298.43,
 247.15,
 235.14]


for e in range(1296,episodes):
    episode_reward = 0
    s = env.reset()
    s = np.reshape(s, [1, nS])

    for time in range(1000):

        # Query next action from learner and perform action
        a = agent.selectAction(s)
        s_prime, r, done, info = env.step(a)

        # Add cumulative reward
        episode_reward += r

        # Reshape new state
        s_prime = np.reshape(s_prime, [1, nS])

        # Add experience to memory
        agent.add_memory(s, a, r, s_prime, done)

        # Set current state to new state
        s = s_prime

        #Perform experience replay if memory length is greater than minibatch length

        if len(agent.memory) > agent.minibatch_size:
            agent.replay()

        # If episode is done, exit loop
        if done:
            agent.target_model_update()
            break


    # epsilon decay
    if agent.epsilon > agent.min_e:
        agent.epsilon *= agent.epsilon_decay
        agent.epsilon = agent.min_e + (agent.max_e - agent.min_e)*np.exp(-agent.epsilon_decay*e)

    # Running average of past 100 episodes
    reward_avg.append(episode_reward)
    print('episode: ', e, ' score: ', '%.2f' % episode_reward, ' avg_score: ', '%.2f' % np.average(reward_avg[-100:]), ' frames: ', time, ' epsilon: ', '%.2f' % agent.epsilon)

    # with open('trained_agent.txt', 'a') as f:
    #     f.write(str(np.average(reward_avg)) + '\n')

env.close()


agent.model.save_weights('weights_DQN_'+str(agent.max_e)+'-'+str(agent.min_e)+'-'+str(agent.epsilon_decay)\
                     +'-'+str(agent.gamma)+'-'+str(agent.minibatch_size)+'-'+str(agent.learning_rate)+'.h5')

episode:  1296  score:  231.40  avg_score:  195.83  frames:  258  epsilon:  0.07
episode:  1297  score:  276.39  avg_score:  196.16  frames:  328  epsilon:  0.07
episode:  1298  score:  250.64  avg_score:  196.03  frames:  337  epsilon:  0.07
episode:  1299  score:  279.74  avg_score:  198.40  frames:  202  epsilon:  0.07
episode:  1300  score:  180.66  avg_score:  197.51  frames:  584  epsilon:  0.07
episode:  1301  score:  257.82  avg_score:  199.83  frames:  297  epsilon:  0.07
episode:  1302  score:  253.48  avg_score:  200.07  frames:  268  epsilon:  0.07
episode:  1303  score:  215.63  avg_score:  199.59  frames:  266  epsilon:  0.07
episode:  1304  score:  266.00  avg_score:  199.70  frames:  199  epsilon:  0.07
episode:  1305  score:  50.95  avg_score:  203.93  frames:  173  epsilon:  0.07
episode:  1306  score:  263.49  avg_score:  206.40  frames:  204  epsilon:  0.07
episode:  1307  score:  296.53  avg_score:  209.04  frames:  269  epsilon:  0.07
episode:  1308  score:  33.75

KeyboardInterrupt: 

In [41]:
reward_avg=[260.39,
 249.95,
 265.79,
 233.66,
 261.21,
 253.85,
 280.72,
 278.8,
 298.21,
 255.76,
 256.29,
 270.51,
 285.1,
 268.92,
 275.22,
 262.38,
 244.21,
 271.11,
 247.02,
 216.66,
 286.94,
 282.25,
 256.29,
 281.3,
 269.76,
 248.96,
 268.45,
 216.47,
 273.39,
 275.06,
 250.31,
 275.99,
 257.99,
 -130.06,
 231.62,
 281.62,
 285.23,
 277.81,
 282.65,
 281.32,
 234.27,
 270.24,
 289.71,
 -5.36,
 244.64,
 250.3,
 -313.06,
 252.85,
 274.53,
 298.43,
 247.15,
 235.14,
 231.4,
 276.39,
 250.64,
 279.74,
 180.66,
 257.82,
 253.48,
 215.63,
 266.0,
 50.95,
 263.49,
 296.53,
 33.75,
 16.66,
 146.34,
 273.11,
 265.4,
 291.51,
 286.35,
 -333.71,
 278.9,
 253.29,
 278.3,
 257.73,
 287.55,
 278.31,
 305.23,
 -431.09,
 -5.72,
 262.13,
 270.38,
 262.6,
 273.13,
 238.69,
 275.4,
 260.0,
 -3.71,
 -10.36,
 264.21,
 282.59,
 220.22,
 263.09,
 248.04,
 242.73,
 229.64,
 285.29,
 86.8,
 220.32]



for e in range(1200,episodes):
    episode_reward = 0
    s = env.reset()
    s = np.reshape(s, [1, nS])

    for time in range(1000):

        # Query next action from learner and perform action
        a = agent.selectAction(s)
        s_prime, r, done, info = env.step(a)

        # Add cumulative reward
        episode_reward += r

        # Reshape new state
        s_prime = np.reshape(s_prime, [1, nS])

        # Add experience to memory
        agent.add_memory(s, a, r, s_prime, done)

        # Set current state to new state
        s = s_prime

        #Perform experience replay if memory length is greater than minibatch length

        if len(agent.memory) > agent.minibatch_size:
            agent.replay()

        # If episode is done, exit loop
        if done:
            agent.target_model_update()
            break


    # epsilon decay
    if agent.epsilon > agent.min_e:
        agent.epsilon *= agent.epsilon_decay
        agent.epsilon = agent.min_e + (agent.max_e - agent.min_e)*np.exp(-agent.epsilon_decay*e)

    # Running average of past 100 episodes
    reward_avg.append(episode_reward)
    print('episode: ', e, ' score: ', '%.2f' % episode_reward, ' avg_score: ', '%.2f' % np.average(reward_avg[-100:]), ' frames: ', time, ' epsilon: ', '%.2f' % agent.epsilon)

    # with open('trained_agent.txt', 'a') as f:
    #     f.write(str(np.average(reward_avg)) + '\n')

env.close()


agent.model.save_weights('weights_DQN_'+str(agent.max_e)+'-'+str(agent.min_e)+'-'+str(agent.epsilon_decay)\
                     +'-'+str(agent.gamma)+'-'+str(agent.minibatch_size)+'-'+str(agent.learning_rate)+'.h5')

episode:  1200  score:  213.20  avg_score:  218.97  frames:  348  epsilon:  0.09
episode:  1201  score:  250.81  avg_score:  218.97  frames:  227  epsilon:  0.09
episode:  1202  score:  256.26  avg_score:  218.88  frames:  468  epsilon:  0.09
episode:  1203  score:  263.66  avg_score:  219.18  frames:  295  epsilon:  0.09
episode:  1204  score:  211.91  avg_score:  218.69  frames:  380  epsilon:  0.09
episode:  1205  score:  -81.38  avg_score:  215.33  frames:  422  epsilon:  0.09
episode:  1206  score:  -124.55  avg_score:  211.28  frames:  79  epsilon:  0.09
episode:  1207  score:  262.29  avg_score:  211.12  frames:  204  epsilon:  0.09
episode:  1208  score:  298.62  avg_score:  211.12  frames:  290  epsilon:  0.09
episode:  1209  score:  56.98  avg_score:  209.13  frames:  169  epsilon:  0.09
episode:  1210  score:  -119.61  avg_score:  205.37  frames:  97  epsilon:  0.09
episode:  1211  score:  260.21  avg_score:  205.27  frames:  342  epsilon:  0.09
episode:  1212  score:  270.5

KeyboardInterrupt: 

In [42]:
'weights_DQN_'+str(agent.max_e)+'-'+str(agent.min_e)+'-'+str(agent.epsilon_decay)\
                         +'-'+str(agent.gamma)+'-'+str(agent.minibatch_size)+'-'+str(agent.learning_rate)+'.h5'

'weights_DQN_1-0.0001-0.002-0.99-30-0.0005.h5'

In [43]:

agent.model.save_weights('weights_DQN_'+str(agent.max_e)+'-'+str(agent.min_e)+'-'+str(agent.epsilon_decay)\
                         +'-'+str(agent.gamma)+'-'+str(agent.minibatch_size)+'-'+str(agent.learning_rate)+'.h5')


In [38]:
string ="""
episode:  0  score:  -217.80  avg_score:  -217.80  frames:  129  epsilon:  1.00
episode:  1  score:  -133.61  avg_score:  -175.70  frames:  104  epsilon:  1.00
episode:  2  score:  -78.27  avg_score:  -143.22  frames:  77  epsilon:  1.00
episode:  3  score:  -100.48  avg_score:  -132.54  frames:  72  epsilon:  0.99
episode:  4  score:  -57.84  avg_score:  -117.60  frames:  67  epsilon:  0.99
episode:  5  score:  -76.59  avg_score:  -110.76  frames:  60  epsilon:  0.99
episode:  6  score:  -311.44  avg_score:  -139.43  frames:  92  epsilon:  0.99
episode:  7  score:  -92.20  avg_score:  -133.53  frames:  91  epsilon:  0.99
episode:  8  score:  -201.70  avg_score:  -141.10  frames:  78  epsilon:  0.98
episode:  9  score:  -137.61  avg_score:  -140.75  frames:  103  epsilon:  0.98
episode:  10  score:  -96.70  avg_score:  -136.75  frames:  138  epsilon:  0.98
episode:  11  score:  -99.51  avg_score:  -133.65  frames:  86  epsilon:  0.98
episode:  12  score:  -110.50  avg_score:  -131.87  frames:  78  epsilon:  0.98
episode:  13  score:  -182.36  avg_score:  -135.47  frames:  129  epsilon:  0.97
episode:  14  score:  -167.63  avg_score:  -137.62  frames:  102  epsilon:  0.97
episode:  15  score:  -87.68  avg_score:  -134.50  frames:  57  epsilon:  0.97
episode:  16  score:  -139.65  avg_score:  -134.80  frames:  88  epsilon:  0.97
episode:  17  score:  -88.13  avg_score:  -132.21  frames:  63  epsilon:  0.97
episode:  18  score:  -137.35  avg_score:  -132.48  frames:  94  epsilon:  0.96
episode:  19  score:  -97.71  avg_score:  -130.74  frames:  67  epsilon:  0.96
episode:  20  score:  -107.11  avg_score:  -129.61  frames:  121  epsilon:  0.96
episode:  21  score:  -249.86  avg_score:  -135.08  frames:  117  epsilon:  0.96
episode:  22  score:  -140.17  avg_score:  -135.30  frames:  63  epsilon:  0.96
episode:  23  score:  -178.72  avg_score:  -137.11  frames:  98  epsilon:  0.96
episode:  24  score:  -373.34  avg_score:  -146.56  frames:  94  epsilon:  0.95
episode:  25  score:  -104.64  avg_score:  -144.95  frames:  69  epsilon:  0.95
episode:  26  score:  -97.10  avg_score:  -143.17  frames:  79  epsilon:  0.95
episode:  27  score:  -348.33  avg_score:  -150.50  frames:  93  epsilon:  0.95
episode:  28  score:  -109.76  avg_score:  -149.10  frames:  84  epsilon:  0.95
episode:  29  score:  -149.88  avg_score:  -149.12  frames:  94  epsilon:  0.94
episode:  30  score:  -272.35  avg_score:  -153.10  frames:  105  epsilon:  0.94
episode:  31  score:  -181.04  avg_score:  -153.97  frames:  110  epsilon:  0.94
episode:  32  score:  -196.41  avg_score:  -155.26  frames:  81  epsilon:  0.94
episode:  33  score:  -273.91  avg_score:  -158.75  frames:  78  epsilon:  0.94
episode:  34  score:  -253.89  avg_score:  -161.47  frames:  149  epsilon:  0.93
episode:  35  score:  -196.66  avg_score:  -162.44  frames:  73  epsilon:  0.93
episode:  36  score:  -205.70  avg_score:  -163.61  frames:  111  epsilon:  0.93
episode:  37  score:  -115.52  avg_score:  -162.35  frames:  64  epsilon:  0.93
episode:  38  score:  -334.27  avg_score:  -166.75  frames:  94  epsilon:  0.93
episode:  39  score:  -189.93  avg_score:  -167.33  frames:  83  epsilon:  0.92
episode:  40  score:  -161.53  avg_score:  -167.19  frames:  115  epsilon:  0.92
episode:  41  score:  -195.18  avg_score:  -167.86  frames:  101  epsilon:  0.92
episode:  42  score:  -97.10  avg_score:  -166.21  frames:  128  epsilon:  0.92
episode:  43  score:  -173.13  avg_score:  -166.37  frames:  78  epsilon:  0.92
episode:  44  score:  -88.10  avg_score:  -164.63  frames:  79  epsilon:  0.92
episode:  45  score:  -111.93  avg_score:  -163.49  frames:  116  epsilon:  0.91
episode:  46  score:  -225.10  avg_score:  -164.80  frames:  128  epsilon:  0.91
episode:  47  score:  -73.52  avg_score:  -162.89  frames:  85  epsilon:  0.91
episode:  48  score:  -105.01  avg_score:  -161.71  frames:  113  epsilon:  0.91
episode:  49  score:  -177.77  avg_score:  -162.03  frames:  82  epsilon:  0.91
episode:  50  score:  -68.87  avg_score:  -160.21  frames:  65  epsilon:  0.90
episode:  51  score:  -161.59  avg_score:  -160.23  frames:  137  epsilon:  0.90
episode:  52  score:  -87.22  avg_score:  -158.86  frames:  69  epsilon:  0.90
episode:  53  score:  -66.59  avg_score:  -157.15  frames:  72  epsilon:  0.90
episode:  54  score:  -91.14  avg_score:  -155.95  frames:  87  epsilon:  0.90
episode:  55  score:  -316.14  avg_score:  -158.81  frames:  70  epsilon:  0.90
episode:  56  score:  27.40  avg_score:  -155.54  frames:  138  epsilon:  0.89
episode:  57  score:  -96.96  avg_score:  -154.53  frames:  113  epsilon:  0.89
episode:  58  score:  -139.55  avg_score:  -154.28  frames:  76  epsilon:  0.89
episode:  59  score:  -121.55  avg_score:  -153.73  frames:  89  epsilon:  0.89
episode:  60  score:  -127.81  avg_score:  -153.31  frames:  70  epsilon:  0.89
episode:  61  score:  -260.42  avg_score:  -155.04  frames:  81  epsilon:  0.89
episode:  62  score:  -49.00  avg_score:  -153.35  frames:  110  epsilon:  0.88
episode:  63  score:  -98.34  avg_score:  -152.49  frames:  74  epsilon:  0.88
episode:  64  score:  -106.51  avg_score:  -151.79  frames:  79  epsilon:  0.88
episode:  65  score:  -95.54  avg_score:  -150.93  frames:  85  epsilon:  0.88
episode:  66  score:  -66.75  avg_score:  -149.68  frames:  75  epsilon:  0.88
episode:  67  score:  -152.28  avg_score:  -149.71  frames:  112  epsilon:  0.87
episode:  68  score:  -161.25  avg_score:  -149.88  frames:  120  epsilon:  0.87
episode:  69  score:  -255.13  avg_score:  -151.39  frames:  124  epsilon:  0.87
episode:  70  score:  -71.69  avg_score:  -150.26  frames:  74  epsilon:  0.87
episode:  71  score:  -383.86  avg_score:  -153.51  frames:  87  epsilon:  0.87
episode:  72  score:  -86.26  avg_score:  -152.59  frames:  75  epsilon:  0.87
episode:  73  score:  -124.38  avg_score:  -152.21  frames:  116  epsilon:  0.86
episode:  74  score:  -260.76  avg_score:  -153.65  frames:  124  epsilon:  0.86
episode:  75  score:  -75.09  avg_score:  -152.62  frames:  93  epsilon:  0.86
episode:  76  score:  -106.39  avg_score:  -152.02  frames:  88  epsilon:  0.86
episode:  77  score:  -57.72  avg_score:  -150.81  frames:  135  epsilon:  0.86
episode:  78  score:  10.92  avg_score:  -148.76  frames:  77  epsilon:  0.86
episode:  79  score:  -223.76  avg_score:  -149.70  frames:  102  epsilon:  0.85
episode:  80  score:  -6.35  avg_score:  -147.93  frames:  109  epsilon:  0.85
episode:  81  score:  -89.77  avg_score:  -147.22  frames:  96  epsilon:  0.85
episode:  82  score:  -121.26  avg_score:  -146.91  frames:  126  epsilon:  0.85
episode:  83  score:  -110.88  avg_score:  -146.48  frames:  132  epsilon:  0.85
episode:  84  score:  -123.69  avg_score:  -146.21  frames:  131  epsilon:  0.85
episode:  85  score:  -88.11  avg_score:  -145.54  frames:  70  epsilon:  0.84
episode:  86  score:  -98.57  avg_score:  -145.00  frames:  87  epsilon:  0.84
episode:  87  score:  -44.72  avg_score:  -143.86  frames:  68  epsilon:  0.84
episode:  88  score:  -83.00  avg_score:  -143.17  frames:  91  epsilon:  0.84
episode:  89  score:  -148.14  avg_score:  -143.23  frames:  116  epsilon:  0.84
episode:  90  score:  -75.68  avg_score:  -142.49  frames:  101  epsilon:  0.84
episode:  91  score:  -131.50  avg_score:  -142.37  frames:  110  epsilon:  0.83
episode:  92  score:  -50.64  avg_score:  -141.38  frames:  144  epsilon:  0.83
episode:  93  score:  -239.53  avg_score:  -142.42  frames:  139  epsilon:  0.83
episode:  94  score:  -149.76  avg_score:  -142.50  frames:  111  epsilon:  0.83
episode:  95  score:  -106.05  avg_score:  -142.12  frames:  86  epsilon:  0.83
episode:  96  score:  -86.41  avg_score:  -141.55  frames:  94  epsilon:  0.83
episode:  97  score:  -89.75  avg_score:  -141.02  frames:  112  epsilon:  0.82
episode:  98  score:  -101.65  avg_score:  -140.62  frames:  95  epsilon:  0.82
episode:  99  score:  -118.30  avg_score:  -140.40  frames:  68  epsilon:  0.82
episode:  100  score:  -124.49  avg_score:  -139.46  frames:  92  epsilon:  0.82
episode:  101  score:  -149.76  avg_score:  -139.63  frames:  107  epsilon:  0.82
episode:  102  score:  -120.09  avg_score:  -140.04  frames:  83  epsilon:  0.82
episode:  103  score:  -92.00  avg_score:  -139.96  frames:  115  epsilon:  0.81
episode:  104  score:  -94.06  avg_score:  -140.32  frames:  74  epsilon:  0.81
episode:  105  score:  -50.45  avg_score:  -140.06  frames:  71  epsilon:  0.81
episode:  106  score:  -190.11  avg_score:  -138.85  frames:  100  epsilon:  0.81
episode:  107  score:  -15.36  avg_score:  -138.08  frames:  122  epsilon:  0.81
episode:  108  score:  -149.08  avg_score:  -137.55  frames:  64  epsilon:  0.81
episode:  109  score:  -69.14  avg_score:  -136.87  frames:  78  epsilon:  0.80
episode:  110  score:  -139.24  avg_score:  -137.29  frames:  81  epsilon:  0.80
episode:  111  score:  -74.54  avg_score:  -137.04  frames:  74  epsilon:  0.80
episode:  112  score:  -47.72  avg_score:  -136.41  frames:  117  epsilon:  0.80
episode:  113  score:  -41.08  avg_score:  -135.00  frames:  71  epsilon:  0.80
episode:  114  score:  -109.89  avg_score:  -134.42  frames:  79  epsilon:  0.80
episode:  115  score:  -145.00  avg_score:  -135.00  frames:  80  epsilon:  0.79
episode:  116  score:  -44.78  avg_score:  -134.05  frames:  72  epsilon:  0.79
episode:  117  score:  -50.02  avg_score:  -133.67  frames:  126  epsilon:  0.79
episode:  118  score:  -74.91  avg_score:  -133.04  frames:  123  epsilon:  0.79
episode:  119  score:  10.68  avg_score:  -131.96  frames:  144  epsilon:  0.79
episode:  120  score:  -112.17  avg_score:  -132.01  frames:  121  epsilon:  0.79
episode:  121  score:  -55.68  avg_score:  -130.07  frames:  67  epsilon:  0.79
episode:  122  score:  -69.95  avg_score:  -129.37  frames:  113  epsilon:  0.78
episode:  123  score:  -75.97  avg_score:  -128.34  frames:  121  epsilon:  0.78
episode:  124  score:  -165.98  avg_score:  -126.27  frames:  128  epsilon:  0.78
episode:  125  score:  -30.27  avg_score:  -125.52  frames:  132  epsilon:  0.78
episode:  126  score:  -89.57  avg_score:  -125.45  frames:  114  epsilon:  0.78
episode:  127  score:  -143.14  avg_score:  -123.39  frames:  99  epsilon:  0.78
episode:  128  score:  -125.74  avg_score:  -123.55  frames:  78  epsilon:  0.77
episode:  129  score:  2.08  avg_score:  -122.03  frames:  71  epsilon:  0.77
episode:  130  score:  -73.28  avg_score:  -120.04  frames:  103  epsilon:  0.77
episode:  131  score:  -106.60  avg_score:  -119.30  frames:  106  epsilon:  0.77
episode:  132  score:  -107.23  avg_score:  -118.41  frames:  104  epsilon:  0.77
episode:  133  score:  -39.93  avg_score:  -116.07  frames:  71  epsilon:  0.77
episode:  134  score:  -75.20  avg_score:  -114.28  frames:  79  epsilon:  0.76
episode:  135  score:  -130.34  avg_score:  -113.62  frames:  81  epsilon:  0.76
episode:  136  score:  -83.07  avg_score:  -112.39  frames:  108  epsilon:  0.76
episode:  137  score:  -106.43  avg_score:  -112.30  frames:  103  epsilon:  0.76
episode:  138  score:  -73.41  avg_score:  -109.69  frames:  121  epsilon:  0.76
episode:  139  score:  -101.77  avg_score:  -108.81  frames:  81  epsilon:  0.76
episode:  140  score:  -84.01  avg_score:  -108.04  frames:  88  epsilon:  0.76
episode:  141  score:  -15.02  avg_score:  -106.23  frames:  85  epsilon:  0.75
episode:  142  score:  -103.31  avg_score:  -106.30  frames:  71  epsilon:  0.75
episode:  143  score:  -98.79  avg_score:  -105.55  frames:  90  epsilon:  0.75
episode:  144  score:  -77.00  avg_score:  -105.44  frames:  93  epsilon:  0.75
episode:  145  score:  -264.97  avg_score:  -106.97  frames:  75  epsilon:  0.75
episode:  146  score:  -73.88  avg_score:  -105.46  frames:  87  epsilon:  0.75
episode:  147  score:  -50.65  avg_score:  -105.23  frames:  132  epsilon:  0.75
episode:  148  score:  -84.11  avg_score:  -105.02  frames:  98  epsilon:  0.74
episode:  149  score:  -106.24  avg_score:  -104.31  frames:  91  epsilon:  0.74
episode:  150  score:  -65.73  avg_score:  -104.28  frames:  138  epsilon:  0.74
episode:  151  score:  -62.26  avg_score:  -103.28  frames:  62  epsilon:  0.74
episode:  152  score:  -48.05  avg_score:  -102.89  frames:  87  epsilon:  0.74
episode:  153  score:  -79.62  avg_score:  -103.02  frames:  119  epsilon:  0.74
episode:  154  score:  -76.70  avg_score:  -102.88  frames:  151  epsilon:  0.73
episode:  155  score:  -102.74  avg_score:  -100.74  frames:  140  epsilon:  0.73
episode:  156  score:  -67.32  avg_score:  -101.69  frames:  104  epsilon:  0.73
episode:  157  score:  -71.47  avg_score:  -101.43  frames:  118  epsilon:  0.73
episode:  158  score:  -68.95  avg_score:  -100.73  frames:  73  epsilon:  0.73
episode:  159  score:  12.20  avg_score:  -99.39  frames:  120  epsilon:  0.73
episode:  160  score:  1.28  avg_score:  -98.10  frames:  80  epsilon:  0.73
episode:  161  score:  -213.82  avg_score:  -97.63  frames:  109  epsilon:  0.72
episode:  162  score:  -32.59  avg_score:  -97.47  frames:  128  epsilon:  0.72
episode:  163  score:  -87.89  avg_score:  -97.37  frames:  93  epsilon:  0.72
episode:  164  score:  -115.88  avg_score:  -97.46  frames:  101  epsilon:  0.72
episode:  165  score:  -50.77  avg_score:  -97.01  frames:  141  epsilon:  0.72
episode:  166  score:  -57.47  avg_score:  -96.92  frames:  86  epsilon:  0.72
episode:  167  score:  -82.89  avg_score:  -96.22  frames:  108  epsilon:  0.72
episode:  168  score:  -37.97  avg_score:  -94.99  frames:  164  epsilon:  0.71
episode:  169  score:  -36.19  avg_score:  -92.80  frames:  124  epsilon:  0.71
episode:  170  score:  -90.94  avg_score:  -93.00  frames:  81  epsilon:  0.71
episode:  171  score:  -65.04  avg_score:  -89.81  frames:  70  epsilon:  0.71
episode:  172  score:  -74.34  avg_score:  -89.69  frames:  117  epsilon:  0.71
episode:  173  score:  -120.15  avg_score:  -89.65  frames:  90  epsilon:  0.71
episode:  174  score:  -70.17  avg_score:  -87.74  frames:  65  epsilon:  0.71
episode:  175  score:  -93.94  avg_score:  -87.93  frames:  122  epsilon:  0.70
episode:  176  score:  -38.32  avg_score:  -87.25  frames:  133  epsilon:  0.70
episode:  177  score:  -37.93  avg_score:  -87.05  frames:  74  epsilon:  0.70
episode:  178  score:  -90.02  avg_score:  -88.06  frames:  78  epsilon:  0.70
episode:  179  score:  -88.16  avg_score:  -86.70  frames:  91  epsilon:  0.70
episode:  180  score:  -39.35  avg_score:  -87.03  frames:  79  epsilon:  0.70
episode:  181  score:  -69.00  avg_score:  -86.82  frames:  146  epsilon:  0.70
episode:  182  score:  -54.04  avg_score:  -86.15  frames:  62  epsilon:  0.69
episode:  183  score:  -100.44  avg_score:  -86.05  frames:  105  epsilon:  0.69
episode:  184  score:  -82.68  avg_score:  -85.64  frames:  92  epsilon:  0.69
episode:  185  score:  -75.37  avg_score:  -85.51  frames:  82  epsilon:  0.69
episode:  186  score:  -57.65  avg_score:  -85.10  frames:  115  epsilon:  0.69
episode:  187  score:  -63.75  avg_score:  -85.29  frames:  124  epsilon:  0.69
episode:  188  score:  -55.07  avg_score:  -85.01  frames:  74  epsilon:  0.69
episode:  189  score:  -122.72  avg_score:  -84.76  frames:  92  epsilon:  0.69
episode:  190  score:  -110.54  avg_score:  -85.11  frames:  99  epsilon:  0.68
episode:  191  score:  -108.71  avg_score:  -84.88  frames:  140  epsilon:  0.68
episode:  192  score:  -5.92  avg_score:  -84.43  frames:  999  epsilon:  0.68
episode:  193  score:  -23.36  avg_score:  -82.27  frames:  116  epsilon:  0.68
episode:  194  score:  -18.43  avg_score:  -80.96  frames:  127  epsilon:  0.68
episode:  195  score:  -73.00  avg_score:  -80.63  frames:  136  epsilon:  0.68
episode:  196  score:  -59.30  avg_score:  -80.36  frames:  92  epsilon:  0.68
episode:  197  score:  -95.42  avg_score:  -80.41  frames:  106  epsilon:  0.67
episode:  198  score:  -116.33  avg_score:  -80.56  frames:  60  epsilon:  0.67
episode:  199  score:  -37.54  avg_score:  -79.75  frames:  121  epsilon:  0.67
episode:  200  score:  -59.59  avg_score:  -79.10  frames:  120  epsilon:  0.67
episode:  201  score:  8.38  avg_score:  -77.52  frames:  146  epsilon:  0.67
episode:  202  score:  -76.78  avg_score:  -77.09  frames:  102  epsilon:  0.67
episode:  203  score:  -120.71  avg_score:  -77.37  frames:  93  epsilon:  0.67
episode:  204  score:  -112.81  avg_score:  -77.56  frames:  121  epsilon:  0.67
episode:  205  score:  -56.77  avg_score:  -77.63  frames:  80  epsilon:  0.66
episode:  206  score:  -68.83  avg_score:  -76.41  frames:  147  epsilon:  0.66
episode:  207  score:  -15.10  avg_score:  -76.41  frames:  112  epsilon:  0.66
episode:  208  score:  -215.05  avg_score:  -77.07  frames:  100  epsilon:  0.66
episode:  209  score:  8.91  avg_score:  -76.29  frames:  166  epsilon:  0.66
episode:  210  score:  -42.44  avg_score:  -75.32  frames:  122  epsilon:  0.66
episode:  211  score:  -40.93  avg_score:  -74.99  frames:  82  epsilon:  0.66
episode:  212  score:  -52.78  avg_score:  -75.04  frames:  104  epsilon:  0.65
episode:  213  score:  -50.21  avg_score:  -75.13  frames:  76  epsilon:  0.65
episode:  214  score:  -55.33  avg_score:  -74.58  frames:  124  epsilon:  0.65
episode:  215  score:  -22.70  avg_score:  -73.36  frames:  149  epsilon:  0.65
episode:  216  score:  -100.64  avg_score:  -73.92  frames:  176  epsilon:  0.65
episode:  217  score:  -63.28  avg_score:  -74.05  frames:  116  epsilon:  0.65
episode:  218  score:  -58.85  avg_score:  -73.89  frames:  162  epsilon:  0.65
episode:  219  score:  -33.38  avg_score:  -74.33  frames:  136  epsilon:  0.65
episode:  220  score:  -189.47  avg_score:  -75.10  frames:  167  epsilon:  0.64
episode:  221  score:  -128.54  avg_score:  -75.83  frames:  121  epsilon:  0.64
episode:  222  score:  -127.02  avg_score:  -76.40  frames:  124  epsilon:  0.64
episode:  223  score:  -63.46  avg_score:  -76.28  frames:  140  epsilon:  0.64
episode:  224  score:  -58.14  avg_score:  -75.20  frames:  140  epsilon:  0.64
episode:  225  score:  -42.86  avg_score:  -75.32  frames:  71  epsilon:  0.64
episode:  226  score:  -64.48  avg_score:  -75.07  frames:  142  epsilon:  0.64
episode:  227  score:  -53.22  avg_score:  -74.17  frames:  121  epsilon:  0.64
episode:  228  score:  -100.30  avg_score:  -73.92  frames:  108  epsilon:  0.63
episode:  229  score:  -45.51  avg_score:  -74.40  frames:  109  epsilon:  0.63
episode:  230  score:  -125.66  avg_score:  -74.92  frames:  100  epsilon:  0.63
episode:  231  score:  -147.17  avg_score:  -75.33  frames:  124  epsilon:  0.63
episode:  232  score:  -67.33  avg_score:  -74.93  frames:  118  epsilon:  0.63
episode:  233  score:  -84.80  avg_score:  -75.38  frames:  124  epsilon:  0.63
episode:  234  score:  -127.96  avg_score:  -75.90  frames:  150  epsilon:  0.63
episode:  235  score:  -91.43  avg_score:  -75.51  frames:  110  epsilon:  0.63
episode:  236  score:  -95.08  avg_score:  -75.63  frames:  136  epsilon:  0.62
episode:  237  score:  -16.08  avg_score:  -74.73  frames:  181  epsilon:  0.62
episode:  238  score:  -99.54  avg_score:  -74.99  frames:  92  epsilon:  0.62
episode:  239  score:  -23.53  avg_score:  -74.21  frames:  143  epsilon:  0.62
episode:  240  score:  -36.90  avg_score:  -73.74  frames:  143  epsilon:  0.62
episode:  241  score:  -72.91  avg_score:  -74.32  frames:  136  epsilon:  0.62
episode:  242  score:  -82.44  avg_score:  -74.11  frames:  94  epsilon:  0.62
episode:  243  score:  -92.51  avg_score:  -74.05  frames:  125  epsilon:  0.62
episode:  244  score:  -44.17  avg_score:  -73.72  frames:  99  epsilon:  0.61
episode:  245  score:  -70.46  avg_score:  -71.77  frames:  126  epsilon:  0.61
episode:  246  score:  -40.56  avg_score:  -71.44  frames:  153  epsilon:  0.61
episode:  247  score:  -40.58  avg_score:  -71.34  frames:  156  epsilon:  0.61
episode:  248  score:  -83.07  avg_score:  -71.33  frames:  135  epsilon:  0.61
episode:  249  score:  -43.81  avg_score:  -70.70  frames:  118  epsilon:  0.61
episode:  250  score:  -40.63  avg_score:  -70.45  frames:  151  epsilon:  0.61
episode:  251  score:  -67.14  avg_score:  -70.50  frames:  166  epsilon:  0.61
episode:  252  score:  -3.43  avg_score:  -70.06  frames:  131  epsilon:  0.60
episode:  253  score:  -73.24  avg_score:  -69.99  frames:  152  epsilon:  0.60
episode:  254  score:  -86.00  avg_score:  -70.08  frames:  145  epsilon:  0.60
episode:  255  score:  -17.99  avg_score:  -69.24  frames:  100  epsilon:  0.60
episode:  256  score:  21.27  avg_score:  -68.35  frames:  110  epsilon:  0.60
episode:  257  score:  -37.00  avg_score:  -68.01  frames:  118  epsilon:  0.60
episode:  258  score:  -83.22  avg_score:  -68.15  frames:  143  epsilon:  0.60
episode:  259  score:  -56.53  avg_score:  -68.84  frames:  115  epsilon:  0.60
episode:  260  score:  -41.06  avg_score:  -69.26  frames:  132  epsilon:  0.59
episode:  261  score:  -58.92  avg_score:  -67.71  frames:  98  epsilon:  0.59
episode:  262  score:  -55.26  avg_score:  -67.94  frames:  138  epsilon:  0.59
episode:  263  score:  -183.09  avg_score:  -68.89  frames:  126  epsilon:  0.59
episode:  264  score:  -51.52  avg_score:  -68.25  frames:  125  epsilon:  0.59
episode:  265  score:  11.40  avg_score:  -67.62  frames:  137  epsilon:  0.59
episode:  266  score:  2.32  avg_score:  -67.03  frames:  138  epsilon:  0.59
episode:  267  score:  -41.36  avg_score:  -66.61  frames:  72  epsilon:  0.59
episode:  268  score:  -39.28  avg_score:  -66.62  frames:  136  epsilon:  0.59
episode:  269  score:  -34.74  avg_score:  -66.61  frames:  106  epsilon:  0.58
episode:  270  score:  -49.37  avg_score:  -66.19  frames:  114  epsilon:  0.58
episode:  271  score:  -14.47  avg_score:  -65.69  frames:  130  epsilon:  0.58
episode:  272  score:  -27.92  avg_score:  -65.22  frames:  81  epsilon:  0.58
episode:  273  score:  -207.02  avg_score:  -66.09  frames:  128  epsilon:  0.58
episode:  274  score:  -187.16  avg_score:  -67.26  frames:  154  epsilon:  0.58
episode:  275  score:  -54.31  avg_score:  -66.87  frames:  114  epsilon:  0.58
episode:  276  score:  -315.47  avg_score:  -69.64  frames:  533  epsilon:  0.58
episode:  277  score:  30.82  avg_score:  -68.95  frames:  166  epsilon:  0.57
episode:  278  score:  -6.45  avg_score:  -68.11  frames:  106  epsilon:  0.57
episode:  279  score:  -21.53  avg_score:  -67.45  frames:  132  epsilon:  0.57
episode:  280  score:  20.40  avg_score:  -66.85  frames:  97  epsilon:  0.57
episode:  281  score:  -12.57  avg_score:  -66.29  frames:  122  epsilon:  0.57
episode:  282  score:  -26.01  avg_score:  -66.01  frames:  88  epsilon:  0.57
episode:  283  score:  -71.68  avg_score:  -65.72  frames:  999  epsilon:  0.57
episode:  284  score:  -43.00  avg_score:  -65.32  frames:  250  epsilon:  0.57
episode:  285  score:  -25.26  avg_score:  -64.82  frames:  150  epsilon:  0.57
episode:  286  score:  -35.51  avg_score:  -64.60  frames:  155  epsilon:  0.56
episode:  287  score:  15.14  avg_score:  -63.81  frames:  134  epsilon:  0.56
episode:  288  score:  1.90  avg_score:  -63.24  frames:  143  epsilon:  0.56
episode:  289  score:  -54.18  avg_score:  -62.56  frames:  117  epsilon:  0.56
episode:  290  score:  -51.65  avg_score:  -61.97  frames:  144  epsilon:  0.56
episode:  291  score:  -74.64  avg_score:  -61.63  frames:  105  epsilon:  0.56
episode:  292  score:  -28.06  avg_score:  -61.85  frames:  89  epsilon:  0.56
episode:  293  score:  -73.09  avg_score:  -62.34  frames:  157  epsilon:  0.56
episode:  294  score:  12.85  avg_score:  -62.03  frames:  121  epsilon:  0.56
episode:  295  score:  -32.66  avg_score:  -61.63  frames:  118  epsilon:  0.55
episode:  296  score:  -74.65  avg_score:  -61.78  frames:  81  epsilon:  0.55
episode:  297  score:  -10.41  avg_score:  -60.93  frames:  107  epsilon:  0.55
episode:  298  score:  -79.89  avg_score:  -60.57  frames:  108  epsilon:  0.55
episode:  299  score:  -159.26  avg_score:  -61.78  frames:  101  epsilon:  0.55
episode:  300  score:  26.77  avg_score:  -60.92  frames:  116  epsilon:  0.55
episode:  301  score:  -36.23  avg_score:  -61.37  frames:  159  epsilon:  0.55
episode:  302  score:  7.90  avg_score:  -60.52  frames:  138  epsilon:  0.55
episode:  303  score:  -39.80  avg_score:  -59.71  frames:  134  epsilon:  0.55
episode:  304  score:  -2.05  avg_score:  -58.60  frames:  122  epsilon:  0.54
episode:  305  score:  2.49  avg_score:  -58.01  frames:  123  epsilon:  0.54
episode:  306  score:  38.66  avg_score:  -56.94  frames:  116  epsilon:  0.54
episode:  307  score:  -15.83  avg_score:  -56.94  frames:  101  epsilon:  0.54
episode:  308  score:  -47.21  avg_score:  -55.26  frames:  119  epsilon:  0.54
episode:  309  score:  -18.93  avg_score:  -55.54  frames:  116  epsilon:  0.54
episode:  310  score:  -42.04  avg_score:  -55.54  frames:  126  epsilon:  0.54
episode:  311  score:  -16.28  avg_score:  -55.29  frames:  82  epsilon:  0.54
episode:  312  score:  -27.05  avg_score:  -55.04  frames:  999  epsilon:  0.54
episode:  313  score:  -28.12  avg_score:  -54.81  frames:  123  epsilon:  0.53
episode:  314  score:  -16.57  avg_score:  -54.43  frames:  138  epsilon:  0.53
episode:  315  score:  -63.74  avg_score:  -54.84  frames:  118  epsilon:  0.53
episode:  316  score:  -52.72  avg_score:  -54.36  frames:  126  epsilon:  0.53
episode:  317  score:  -56.12  avg_score:  -54.29  frames:  107  epsilon:  0.53
episode:  318  score:  -67.97  avg_score:  -54.38  frames:  102  epsilon:  0.53
episode:  319  score:  -41.74  avg_score:  -54.46  frames:  160  epsilon:  0.53
episode:  320  score:  -11.76  avg_score:  -52.68  frames:  140  epsilon:  0.53
episode:  321  score:  -211.66  avg_score:  -53.52  frames:  147  epsilon:  0.53
episode:  322  score:  -77.51  avg_score:  -53.02  frames:  139  epsilon:  0.53
episode:  323  score:  -10.01  avg_score:  -52.49  frames:  118  epsilon:  0.52
episode:  324  score:  -45.89  avg_score:  -52.36  frames:  88  epsilon:  0.52
episode:  325  score:  -3.44  avg_score:  -51.97  frames:  120  epsilon:  0.52
episode:  326  score:  -9.56  avg_score:  -51.42  frames:  137  epsilon:  0.52
episode:  327  score:  -12.82  avg_score:  -51.02  frames:  999  epsilon:  0.52
episode:  328  score:  -3.66  avg_score:  -50.05  frames:  102  epsilon:  0.52
episode:  329  score:  -16.27  avg_score:  -49.76  frames:  153  epsilon:  0.52
episode:  330  score:  26.43  avg_score:  -48.24  frames:  550  epsilon:  0.52
episode:  331  score:  -54.01  avg_score:  -47.30  frames:  92  epsilon:  0.52
episode:  332  score:  -18.32  avg_score:  -46.81  frames:  138  epsilon:  0.51
episode:  333  score:  -40.60  avg_score:  -46.37  frames:  138  epsilon:  0.51
episode:  334  score:  -56.18  avg_score:  -45.65  frames:  121  epsilon:  0.51
episode:  335  score:  -42.32  avg_score:  -45.16  frames:  124  epsilon:  0.51
episode:  336  score:  40.88  avg_score:  -43.80  frames:  168  epsilon:  0.51
episode:  337  score:  -37.67  avg_score:  -44.02  frames:  114  epsilon:  0.51
episode:  338  score:  -54.38  avg_score:  -43.57  frames:  134  epsilon:  0.51
episode:  339  score:  -34.94  avg_score:  -43.68  frames:  86  epsilon:  0.51
episode:  340  score:  -2.52  avg_score:  -43.34  frames:  122  epsilon:  0.51
episode:  341  score:  -26.45  avg_score:  -42.87  frames:  196  epsilon:  0.51
episode:  342  score:  -173.14  avg_score:  -43.78  frames:  178  epsilon:  0.50
episode:  343  score:  -47.11  avg_score:  -43.33  frames:  108  epsilon:  0.50
episode:  344  score:  -26.52  avg_score:  -43.15  frames:  126  epsilon:  0.50
episode:  345  score:  -28.50  avg_score:  -42.73  frames:  93  epsilon:  0.50
episode:  346  score:  82.69  avg_score:  -41.50  frames:  999  epsilon:  0.50
episode:  347  score:  -78.92  avg_score:  -41.88  frames:  999  epsilon:  0.50
episode:  348  score:  -18.89  avg_score:  -41.24  frames:  127  epsilon:  0.50
episode:  349  score:  37.65  avg_score:  -40.42  frames:  156  epsilon:  0.50
episode:  350  score:  14.68  avg_score:  -39.87  frames:  132  epsilon:  0.50
episode:  351  score:  -22.94  avg_score:  -39.43  frames:  122  epsilon:  0.50
episode:  352  score:  -13.57  avg_score:  -39.53  frames:  109  epsilon:  0.49
episode:  353  score:  -121.49  avg_score:  -40.01  frames:  156  epsilon:  0.49
episode:  354  score:  -90.84  avg_score:  -40.06  frames:  205  epsilon:  0.49
episode:  355  score:  -25.72  avg_score:  -40.14  frames:  117  epsilon:  0.49
episode:  356  score:  -7.68  avg_score:  -40.43  frames:  111  epsilon:  0.49
episode:  357  score:  -20.00  avg_score:  -40.26  frames:  98  epsilon:  0.49
episode:  358  score:  45.23  avg_score:  -38.97  frames:  126  epsilon:  0.49
episode:  359  score:  -140.81  avg_score:  -39.82  frames:  535  epsilon:  0.49
episode:  360  score:  -9.62  avg_score:  -39.50  frames:  104  epsilon:  0.49
episode:  361  score:  -16.62  avg_score:  -39.08  frames:  100  epsilon:  0.49
episode:  362  score:  -283.05  avg_score:  -41.36  frames:  773  epsilon:  0.48
episode:  363  score:  -101.88  avg_score:  -40.55  frames:  518  epsilon:  0.48
episode:  364  score:  -31.74  avg_score:  -40.35  frames:  119  epsilon:  0.48
episode:  365  score:  -128.03  avg_score:  -41.74  frames:  186  epsilon:  0.48
episode:  366  score:  -67.82  avg_score:  -42.44  frames:  89  epsilon:  0.48
episode:  367  score:  -8.81  avg_score:  -42.12  frames:  152  epsilon:  0.48
episode:  368  score:  -47.95  avg_score:  -42.21  frames:  107  epsilon:  0.48
episode:  369  score:  -26.38  avg_score:  -42.12  frames:  115  epsilon:  0.48
episode:  370  score:  -64.65  avg_score:  -42.27  frames:  71  epsilon:  0.48
episode:  371  score:  14.15  avg_score:  -41.99  frames:  110  epsilon:  0.48
episode:  372  score:  -30.20  avg_score:  -42.01  frames:  999  epsilon:  0.48
episode:  373  score:  -13.08  avg_score:  -40.07  frames:  128  epsilon:  0.47
episode:  374  score:  42.97  avg_score:  -37.77  frames:  105  epsilon:  0.47
episode:  375  score:  -29.26  avg_score:  -37.52  frames:  74  epsilon:  0.47
episode:  376  score:  -52.84  avg_score:  -34.89  frames:  152  epsilon:  0.47
episode:  377  score:  -376.50  avg_score:  -38.97  frames:  345  epsilon:  0.47
episode:  378  score:  -162.44  avg_score:  -40.53  frames:  951  epsilon:  0.47
episode:  379  score:  6.07  avg_score:  -40.25  frames:  91  epsilon:  0.47
episode:  380  score:  3.60  avg_score:  -40.42  frames:  149  epsilon:  0.47
episode:  381  score:  53.27  avg_score:  -39.76  frames:  120  epsilon:  0.47
episode:  382  score:  11.07  avg_score:  -39.39  frames:  183  epsilon:  0.47
episode:  383  score:  -38.52  avg_score:  -39.06  frames:  135  epsilon:  0.46
episode:  384  score:  14.03  avg_score:  -38.49  frames:  89  epsilon:  0.46
episode:  385  score:  -21.84  avg_score:  -38.45  frames:  123  epsilon:  0.46
episode:  386  score:  -2.82  avg_score:  -38.13  frames:  96  epsilon:  0.46
episode:  387  score:  -50.66  avg_score:  -38.78  frames:  112  epsilon:  0.46
episode:  388  score:  -46.27  avg_score:  -39.27  frames:  107  epsilon:  0.46
episode:  389  score:  59.78  avg_score:  -38.13  frames:  999  epsilon:  0.46
episode:  390  score:  -13.24  avg_score:  -37.74  frames:  999  epsilon:  0.46
episode:  391  score:  27.85  avg_score:  -36.72  frames:  85  epsilon:  0.46
episode:  392  score:  63.48  avg_score:  -35.80  frames:  999  epsilon:  0.46
episode:  393  score:  -14.26  avg_score:  -35.21  frames:  161  epsilon:  0.46
episode:  394  score:  -69.62  avg_score:  -36.04  frames:  99  epsilon:  0.45
episode:  395  score:  -23.06  avg_score:  -35.94  frames:  999  epsilon:  0.45
episode:  396  score:  -20.69  avg_score:  -35.40  frames:  112  epsilon:  0.45
episode:  397  score:  -106.32  avg_score:  -36.36  frames:  395  epsilon:  0.45
episode:  398  score:  -35.72  avg_score:  -35.92  frames:  125  epsilon:  0.45
episode:  399  score:  93.89  avg_score:  -33.39  frames:  999  epsilon:  0.45
episode:  400  score:  -47.81  avg_score:  -34.13  frames:  92  epsilon:  0.45
episode:  401  score:  -16.54  avg_score:  -33.94  frames:  88  epsilon:  0.45
episode:  402  score:  -68.51  avg_score:  -34.70  frames:  116  epsilon:  0.45
episode:  403  score:  -14.18  avg_score:  -34.45  frames:  126  epsilon:  0.45
episode:  404  score:  -77.57  avg_score:  -35.20  frames:  122  epsilon:  0.45
episode:  405  score:  -7.58  avg_score:  -35.30  frames:  95  epsilon:  0.44
episode:  406  score:  -10.51  avg_score:  -35.79  frames:  158  epsilon:  0.44
episode:  407  score:  -102.78  avg_score:  -36.66  frames:  116  epsilon:  0.44
episode:  408  score:  5.49  avg_score:  -36.14  frames:  999  epsilon:  0.44
episode:  409  score:  8.45  avg_score:  -35.86  frames:  152  epsilon:  0.44
episode:  410  score:  -38.56  avg_score:  -35.83  frames:  115  epsilon:  0.44
episode:  411  score:  -57.47  avg_score:  -36.24  frames:  104  epsilon:  0.44
episode:  412  score:  131.87  avg_score:  -34.65  frames:  999  epsilon:  0.44
episode:  413  score:  54.87  avg_score:  -33.82  frames:  999  epsilon:  0.44
episode:  414  score:  14.75  avg_score:  -33.51  frames:  118  epsilon:  0.44
episode:  415  score:  6.00  avg_score:  -32.81  frames:  114  epsilon:  0.44
episode:  416  score:  -44.94  avg_score:  -32.73  frames:  159  epsilon:  0.44
episode:  417  score:  -33.45  avg_score:  -32.50  frames:  125  epsilon:  0.43
episode:  418  score:  -22.84  avg_score:  -32.05  frames:  111  epsilon:  0.43
episode:  419  score:  -22.83  avg_score:  -31.86  frames:  130  epsilon:  0.43
episode:  420  score:  -205.06  avg_score:  -33.80  frames:  166  epsilon:  0.43
episode:  421  score:  57.30  avg_score:  -31.11  frames:  999  epsilon:  0.43
episode:  422  score:  -116.87  avg_score:  -31.50  frames:  999  epsilon:  0.43
episode:  423  score:  24.81  avg_score:  -31.15  frames:  143  epsilon:  0.43
episode:  424  score:  108.86  avg_score:  -29.61  frames:  999  epsilon:  0.43
episode:  425  score:  -46.58  avg_score:  -30.04  frames:  160  epsilon:  0.43
episode:  426  score:  121.40  avg_score:  -28.73  frames:  999  epsilon:  0.43
episode:  427  score:  12.71  avg_score:  -28.47  frames:  140  epsilon:  0.43
episode:  428  score:  52.93  avg_score:  -27.91  frames:  999  epsilon:  0.42
episode:  429  score:  37.57  avg_score:  -27.37  frames:  126  epsilon:  0.42
episode:  430  score:  -183.90  avg_score:  -29.47  frames:  120  epsilon:  0.42
episode:  431  score:  -41.17  avg_score:  -29.34  frames:  125  epsilon:  0.42
episode:  432  score:  0.46  avg_score:  -29.15  frames:  121  epsilon:  0.42
episode:  433  score:  -70.78  avg_score:  -29.46  frames:  146  epsilon:  0.42
episode:  434  score:  -9.40  avg_score:  -28.99  frames:  999  epsilon:  0.42
episode:  435  score:  -28.48  avg_score:  -28.85  frames:  96  epsilon:  0.42
episode:  436  score:  118.11  avg_score:  -28.08  frames:  999  epsilon:  0.42
episode:  437  score:  12.27  avg_score:  -27.58  frames:  119  epsilon:  0.42
episode:  438  score:  -225.36  avg_score:  -29.29  frames:  431  epsilon:  0.42
episode:  439  score:  -39.03  avg_score:  -29.33  frames:  98  epsilon:  0.42
episode:  440  score:  -269.42  avg_score:  -32.00  frames:  646  epsilon:  0.41
episode:  441  score:  3.04  avg_score:  -31.70  frames:  138  epsilon:  0.41
episode:  442  score:  -177.06  avg_score:  -31.74  frames:  200  epsilon:  0.41
episode:  443  score:  50.41  avg_score:  -30.77  frames:  999  epsilon:  0.41
episode:  444  score:  -64.64  avg_score:  -31.15  frames:  113  epsilon:  0.41
episode:  445  score:  -355.07  avg_score:  -34.41  frames:  904  epsilon:  0.41
episode:  446  score:  -135.56  avg_score:  -36.60  frames:  166  epsilon:  0.41
episode:  447  score:  -31.49  avg_score:  -36.12  frames:  100  epsilon:  0.41
episode:  448  score:  90.36  avg_score:  -35.03  frames:  999  epsilon:  0.41
episode:  449  score:  134.39  avg_score:  -34.06  frames:  999  epsilon:  0.41
episode:  450  score:  -0.24  avg_score:  -34.21  frames:  245  epsilon:  0.41
episode:  451  score:  73.61  avg_score:  -33.25  frames:  999  epsilon:  0.41
episode:  452  score:  8.69  avg_score:  -33.02  frames:  177  epsilon:  0.41
episode:  453  score:  -30.62  avg_score:  -32.11  frames:  999  epsilon:  0.40
episode:  454  score:  50.81  avg_score:  -30.70  frames:  999  epsilon:  0.40
episode:  455  score:  -67.84  avg_score:  -31.12  frames:  373  epsilon:  0.40
episode:  456  score:  -8.26  avg_score:  -31.13  frames:  80  epsilon:  0.40
episode:  457  score:  109.62  avg_score:  -29.83  frames:  999  epsilon:  0.40
episode:  458  score:  66.72  avg_score:  -29.61  frames:  999  epsilon:  0.40
episode:  459  score:  104.63  avg_score:  -27.16  frames:  999  epsilon:  0.40
episode:  460  score:  28.77  avg_score:  -26.78  frames:  151  epsilon:  0.40
episode:  461  score:  -275.20  avg_score:  -29.36  frames:  822  epsilon:  0.40
episode:  462  score:  1.22  avg_score:  -26.52  frames:  97  epsilon:  0.40
episode:  463  score:  -43.56  avg_score:  -25.94  frames:  132  epsilon:  0.40
episode:  464  score:  18.08  avg_score:  -25.44  frames:  146  epsilon:  0.40
episode:  465  score:  80.93  avg_score:  -23.35  frames:  999  epsilon:  0.39
episode:  466  score:  90.30  avg_score:  -21.77  frames:  999  epsilon:  0.39
episode:  467  score:  -28.68  avg_score:  -21.97  frames:  273  epsilon:  0.39
episode:  468  score:  26.88  avg_score:  -21.22  frames:  999  epsilon:  0.39
episode:  469  score:  55.58  avg_score:  -20.40  frames:  999  epsilon:  0.39
episode:  470  score:  -181.65  avg_score:  -21.57  frames:  146  epsilon:  0.39
episode:  471  score:  -210.77  avg_score:  -23.82  frames:  187  epsilon:  0.39
episode:  472  score:  60.89  avg_score:  -22.91  frames:  999  epsilon:  0.39
episode:  473  score:  -29.31  avg_score:  -23.07  frames:  73  epsilon:  0.39
episode:  474  score:  -162.98  avg_score:  -25.13  frames:  581  epsilon:  0.39
episode:  475  score:  105.87  avg_score:  -23.78  frames:  999  epsilon:  0.39
episode:  476  score:  50.65  avg_score:  -22.74  frames:  999  epsilon:  0.39
episode:  477  score:  -42.88  avg_score:  -19.41  frames:  106  epsilon:  0.39
episode:  478  score:  -27.95  avg_score:  -18.06  frames:  145  epsilon:  0.38
episode:  479  score:  -26.46  avg_score:  -18.39  frames:  102  epsilon:  0.38
episode:  480  score:  -12.15  avg_score:  -18.54  frames:  95  epsilon:  0.38
episode:  481  score:  31.80  avg_score:  -18.76  frames:  139  epsilon:  0.38
episode:  482  score:  -226.32  avg_score:  -21.13  frames:  250  epsilon:  0.38
episode:  483  score:  -17.10  avg_score:  -20.92  frames:  86  epsilon:  0.38
episode:  484  score:  -40.04  avg_score:  -21.46  frames:  113  epsilon:  0.38
episode:  485  score:  -0.95  avg_score:  -21.25  frames:  141  epsilon:  0.38
episode:  486  score:  -7.33  avg_score:  -21.29  frames:  999  epsilon:  0.38
episode:  487  score:  -27.83  avg_score:  -21.07  frames:  295  epsilon:  0.38
episode:  488  score:  33.02  avg_score:  -20.27  frames:  999  epsilon:  0.38
episode:  489  score:  -2.97  avg_score:  -20.90  frames:  124  epsilon:  0.38
episode:  490  score:  33.51  avg_score:  -20.43  frames:  173  epsilon:  0.38
episode:  491  score:  -57.96  avg_score:  -21.29  frames:  365  epsilon:  0.37
episode:  492  score:  -62.82  avg_score:  -22.55  frames:  149  epsilon:  0.37
episode:  493  score:  -151.18  avg_score:  -23.92  frames:  203  epsilon:  0.37
episode:  494  score:  -65.03  avg_score:  -23.88  frames:  149  epsilon:  0.37
episode:  495  score:  82.04  avg_score:  -22.83  frames:  999  epsilon:  0.37
episode:  496  score:  -151.49  avg_score:  -24.13  frames:  311  epsilon:  0.37
episode:  497  score:  52.33  avg_score:  -22.55  frames:  999  epsilon:  0.37
episode:  498  score:  102.83  avg_score:  -21.16  frames:  999  epsilon:  0.37
episode:  499  score:  -60.96  avg_score:  -22.71  frames:  92  epsilon:  0.37
episode:  500  score:  -183.55  avg_score:  -24.07  frames:  251  epsilon:  0.37
episode:  501  score:  6.97  avg_score:  -23.83  frames:  127  epsilon:  0.37
episode:  502  score:  -159.48  avg_score:  -24.74  frames:  987  epsilon:  0.37
episode:  503  score:  115.86  avg_score:  -23.44  frames:  999  epsilon:  0.37
episode:  504  score:  58.40  avg_score:  -22.08  frames:  999  epsilon:  0.37
episode:  505  score:  -93.97  avg_score:  -22.95  frames:  130  epsilon:  0.36
episode:  506  score:  -1.58  avg_score:  -22.86  frames:  99  epsilon:  0.36
episode:  507  score:  23.22  avg_score:  -21.60  frames:  146  epsilon:  0.36
episode:  508  score:  48.35  avg_score:  -21.17  frames:  999  epsilon:  0.36
episode:  509  score:  137.04  avg_score:  -19.88  frames:  999  epsilon:  0.36
episode:  510  score:  -28.85  avg_score:  -19.79  frames:  106  epsilon:  0.36
episode:  511  score:  37.11  avg_score:  -18.84  frames:  118  epsilon:  0.36
episode:  512  score:  42.51  avg_score:  -19.73  frames:  999  epsilon:  0.36
episode:  513  score:  82.60  avg_score:  -19.46  frames:  999  epsilon:  0.36
episode:  514  score:  -74.50  avg_score:  -20.35  frames:  186  epsilon:  0.36
episode:  515  score:  45.35  avg_score:  -19.96  frames:  153  epsilon:  0.36
episode:  516  score:  35.58  avg_score:  -19.15  frames:  99  epsilon:  0.36
episode:  517  score:  -20.71  avg_score:  -19.02  frames:  137  epsilon:  0.36
episode:  518  score:  4.65  avg_score:  -18.75  frames:  154  epsilon:  0.35
episode:  519  score:  120.43  avg_score:  -17.32  frames:  999  epsilon:  0.35
episode:  520  score:  127.31  avg_score:  -13.99  frames:  999  epsilon:  0.35
episode:  521  score:  -157.68  avg_score:  -16.14  frames:  153  epsilon:  0.35
episode:  522  score:  129.53  avg_score:  -13.68  frames:  999  epsilon:  0.35
episode:  523  score:  -8.63  avg_score:  -14.01  frames:  104  epsilon:  0.35
episode:  524  score:  83.55  avg_score:  -14.26  frames:  999  epsilon:  0.35
episode:  525  score:  41.36  avg_score:  -13.39  frames:  999  epsilon:  0.35
episode:  526  score:  102.16  avg_score:  -13.58  frames:  999  epsilon:  0.35
episode:  527  score:  -15.00  avg_score:  -13.86  frames:  109  epsilon:  0.35
episode:  528  score:  -91.29  avg_score:  -15.30  frames:  130  epsilon:  0.35
episode:  529  score:  292.93  avg_score:  -12.74  frames:  493  epsilon:  0.35
episode:  530  score:  -21.98  avg_score:  -11.12  frames:  283  epsilon:  0.35
episode:  531  score:  136.30  avg_score:  -9.35  frames:  999  epsilon:  0.35
episode:  532  score:  105.34  avg_score:  -8.30  frames:  999  epsilon:  0.35
episode:  533  score:  76.63  avg_score:  -6.83  frames:  999  epsilon:  0.34
episode:  534  score:  -36.10  avg_score:  -7.09  frames:  103  epsilon:  0.34
episode:  535  score:  -14.93  avg_score:  -6.96  frames:  104  epsilon:  0.34
episode:  536  score:  121.43  avg_score:  -6.93  frames:  999  epsilon:  0.34
episode:  537  score:  158.41  avg_score:  -5.46  frames:  999  epsilon:  0.34
episode:  538  score:  -53.60  avg_score:  -3.75  frames:  85  epsilon:  0.34
episode:  539  score:  80.36  avg_score:  -2.55  frames:  999  epsilon:  0.34
episode:  540  score:  111.44  avg_score:  1.26  frames:  999  epsilon:  0.34
episode:  541  score:  95.46  avg_score:  2.18  frames:  999  epsilon:  0.34
episode:  542  score:  122.10  avg_score:  5.17  frames:  999  epsilon:  0.34
episode:  543  score:  243.17  avg_score:  7.10  frames:  577  epsilon:  0.34
episode:  544  score:  -318.12  avg_score:  4.56  frames:  296  epsilon:  0.34
episode:  545  score:  191.11  avg_score:  10.03  frames:  240  epsilon:  0.34
episode:  546  score:  137.39  avg_score:  12.76  frames:  999  epsilon:  0.34
episode:  547  score:  -55.29  avg_score:  12.52  frames:  223  epsilon:  0.33
episode:  548  score:  113.92  avg_score:  12.75  frames:  999  epsilon:  0.33
episode:  549  score:  93.48  avg_score:  12.34  frames:  999  epsilon:  0.33
episode:  550  score:  59.31  avg_score:  12.94  frames:  999  epsilon:  0.33
episode:  551  score:  112.92  avg_score:  13.33  frames:  999  epsilon:  0.33
episode:  552  score:  -36.23  avg_score:  12.88  frames:  103  epsilon:  0.33
episode:  553  score:  -11.91  avg_score:  13.07  frames:  112  epsilon:  0.33
episode:  554  score:  139.49  avg_score:  13.96  frames:  999  epsilon:  0.33
episode:  555  score:  13.34  avg_score:  14.77  frames:  159  epsilon:  0.33
episode:  556  score:  139.08  avg_score:  16.24  frames:  999  epsilon:  0.33
episode:  557  score:  12.73  avg_score:  15.27  frames:  157  epsilon:  0.33
episode:  558  score:  109.68  avg_score:  15.70  frames:  999  epsilon:  0.33
episode:  559  score:  2.52  avg_score:  14.68  frames:  80  epsilon:  0.33
episode:  560  score:  9.32  avg_score:  14.49  frames:  142  epsilon:  0.33
episode:  561  score:  47.13  avg_score:  17.71  frames:  193  epsilon:  0.33
episode:  562  score:  63.07  avg_score:  18.33  frames:  167  epsilon:  0.33
episode:  563  score:  15.28  avg_score:  18.92  frames:  155  epsilon:  0.32
episode:  564  score:  5.96  avg_score:  18.80  frames:  80  epsilon:  0.32
episode:  565  score:  73.14  avg_score:  18.72  frames:  999  epsilon:  0.32
episode:  566  score:  15.00  avg_score:  17.97  frames:  139  epsilon:  0.32
episode:  567  score:  131.65  avg_score:  19.57  frames:  999  epsilon:  0.32
episode:  568  score:  -38.54  avg_score:  18.92  frames:  123  epsilon:  0.32
episode:  569  score:  -24.10  avg_score:  18.12  frames:  137  epsilon:  0.32
episode:  570  score:  122.00  avg_score:  21.16  frames:  999  epsilon:  0.32
episode:  571  score:  16.48  avg_score:  23.43  frames:  122  epsilon:  0.32
episode:  572  score:  -18.35  avg_score:  22.64  frames:  143  epsilon:  0.32
episode:  573  score:  25.19  avg_score:  23.18  frames:  120  epsilon:  0.32
episode:  574  score:  23.37  avg_score:  25.04  frames:  999  epsilon:  0.32
episode:  575  score:  -15.61  avg_score:  23.83  frames:  100  epsilon:  0.32
episode:  576  score:  5.50  avg_score:  23.38  frames:  136  epsilon:  0.32
episode:  577  score:  14.43  avg_score:  23.95  frames:  130  epsilon:  0.32
episode:  578  score:  -116.52  avg_score:  23.07  frames:  156  epsilon:  0.31
episode:  579  score:  -304.94  avg_score:  20.28  frames:  192  epsilon:  0.31
episode:  580  score:  2.41  avg_score:  20.43  frames:  114  epsilon:  0.31
episode:  581  score:  -160.71  avg_score:  18.50  frames:  142  epsilon:  0.31
episode:  582  score:  -7.03  avg_score:  20.69  frames:  999  epsilon:  0.31
episode:  583  score:  -2.13  avg_score:  20.84  frames:  999  epsilon:  0.31
episode:  584  score:  151.26  avg_score:  22.76  frames:  999  epsilon:  0.31
episode:  585  score:  31.59  avg_score:  23.08  frames:  90  epsilon:  0.31
episode:  586  score:  14.29  avg_score:  23.30  frames:  91  epsilon:  0.31
episode:  587  score:  136.24  avg_score:  24.94  frames:  999  epsilon:  0.31
episode:  588  score:  115.30  avg_score:  25.76  frames:  999  epsilon:  0.31
episode:  589  score:  -328.64  avg_score:  22.50  frames:  235  epsilon:  0.31
episode:  590  score:  130.02  avg_score:  23.47  frames:  999  epsilon:  0.31
episode:  591  score:  117.02  avg_score:  25.22  frames:  999  epsilon:  0.31
episode:  592  score:  -175.57  avg_score:  24.09  frames:  351  epsilon:  0.31
episode:  593  score:  14.63  avg_score:  25.75  frames:  118  epsilon:  0.31
episode:  594  score:  101.52  avg_score:  27.42  frames:  999  epsilon:  0.30
episode:  595  score:  239.87  avg_score:  28.99  frames:  459  epsilon:  0.30
episode:  596  score:  8.08  avg_score:  30.59  frames:  88  epsilon:  0.30
episode:  597  score:  9.14  avg_score:  30.16  frames:  95  epsilon:  0.30
episode:  598  score:  30.52  avg_score:  29.43  frames:  138  epsilon:  0.30
episode:  599  score:  0.80  avg_score:  30.05  frames:  160  epsilon:  0.30
episode:  600  score:  -0.39  avg_score:  31.88  frames:  123  epsilon:  0.30
episode:  601  score:  -327.07  avg_score:  28.54  frames:  336  epsilon:  0.30
episode:  602  score:  -130.09  avg_score:  28.84  frames:  491  epsilon:  0.30
episode:  603  score:  17.03  avg_score:  27.85  frames:  144  epsilon:  0.30
episode:  604  score:  -443.94  avg_score:  22.83  frames:  224  epsilon:  0.30
episode:  605  score:  -99.82  avg_score:  22.77  frames:  124  epsilon:  0.30
episode:  606  score:  229.58  avg_score:  25.08  frames:  913  epsilon:  0.30
episode:  607  score:  4.57  avg_score:  24.89  frames:  109  epsilon:  0.30
episode:  608  score:  151.22  avg_score:  25.92  frames:  999  epsilon:  0.30
episode:  609  score:  -293.16  avg_score:  21.62  frames:  120  epsilon:  0.30
episode:  610  score:  -144.29  avg_score:  20.46  frames:  602  epsilon:  0.30
episode:  611  score:  -114.85  avg_score:  18.94  frames:  113  epsilon:  0.29
episode:  612  score:  2.76  avg_score:  18.55  frames:  105  epsilon:  0.29
episode:  613  score:  69.60  avg_score:  18.42  frames:  999  epsilon:  0.29
episode:  614  score:  177.98  avg_score:  20.94  frames:  999  epsilon:  0.29
episode:  615  score:  -60.54  avg_score:  19.88  frames:  283  epsilon:  0.29
episode:  616  score:  -88.19  avg_score:  18.65  frames:  127  epsilon:  0.29
episode:  617  score:  -19.29  avg_score:  18.66  frames:  999  epsilon:  0.29
episode:  618  score:  20.36  avg_score:  18.82  frames:  175  epsilon:  0.29
episode:  619  score:  117.80  avg_score:  18.79  frames:  999  epsilon:  0.29
episode:  620  score:  -2.29  avg_score:  17.49  frames:  143  epsilon:  0.29
episode:  621  score:  -387.01  avg_score:  15.20  frames:  883  epsilon:  0.29
episode:  622  score:  28.89  avg_score:  14.19  frames:  999  epsilon:  0.29
episode:  623  score:  -55.27  avg_score:  13.73  frames:  231  epsilon:  0.29
episode:  624  score:  16.87  avg_score:  13.06  frames:  130  epsilon:  0.29
episode:  625  score:  162.34  avg_score:  14.27  frames:  999  epsilon:  0.29
episode:  626  score:  118.93  avg_score:  14.44  frames:  999  epsilon:  0.29
episode:  627  score:  -52.45  avg_score:  14.06  frames:  999  epsilon:  0.29
episode:  628  score:  -285.33  avg_score:  12.12  frames:  255  epsilon:  0.28
episode:  629  score:  159.10  avg_score:  10.79  frames:  999  epsilon:  0.28
episode:  630  score:  10.57  avg_score:  11.11  frames:  184  epsilon:  0.28
episode:  631  score:  21.96  avg_score:  9.97  frames:  207  epsilon:  0.28
episode:  632  score:  155.98  avg_score:  10.47  frames:  999  epsilon:  0.28
episode:  633  score:  99.82  avg_score:  10.71  frames:  999  epsilon:  0.28
episode:  634  score:  -1.90  avg_score:  11.05  frames:  87  epsilon:  0.28
episode:  635  score:  131.67  avg_score:  12.51  frames:  999  epsilon:  0.28
episode:  636  score:  6.78  avg_score:  11.37  frames:  317  epsilon:  0.28
episode:  637  score:  116.55  avg_score:  10.95  frames:  999  epsilon:  0.28
episode:  638  score:  -348.06  avg_score:  8.00  frames:  122  epsilon:  0.28
episode:  639  score:  157.68  avg_score:  8.78  frames:  999  epsilon:  0.28
episode:  640  score:  68.93  avg_score:  8.35  frames:  999  epsilon:  0.28
episode:  641  score:  3.75  avg_score:  7.44  frames:  80  epsilon:  0.28
episode:  642  score:  47.20  avg_score:  6.69  frames:  132  epsilon:  0.28
episode:  643  score:  165.37  avg_score:  5.91  frames:  999  epsilon:  0.28
episode:  644  score:  147.23  avg_score:  10.56  frames:  999  epsilon:  0.28
episode:  645  score:  -1.20  avg_score:  8.64  frames:  78  epsilon:  0.28
episode:  646  score:  118.29  avg_score:  8.45  frames:  999  epsilon:  0.27
episode:  647  score:  251.53  avg_score:  11.52  frames:  482  epsilon:  0.27
episode:  648  score:  -136.83  avg_score:  9.01  frames:  792  epsilon:  0.27
episode:  649  score:  123.16  avg_score:  9.31  frames:  999  epsilon:  0.27
episode:  650  score:  -26.45  avg_score:  8.45  frames:  116  epsilon:  0.27
episode:  651  score:  -18.76  avg_score:  7.13  frames:  129  epsilon:  0.27
episode:  652  score:  46.08  avg_score:  7.95  frames:  176  epsilon:  0.27
episode:  653  score:  13.26  avg_score:  8.21  frames:  178  epsilon:  0.27
episode:  654  score:  19.16  avg_score:  7.00  frames:  121  epsilon:  0.27
episode:  655  score:  -4.00  avg_score:  6.83  frames:  103  epsilon:  0.27
episode:  656  score:  7.02  avg_score:  5.51  frames:  143  epsilon:  0.27
episode:  657  score:  -5.42  avg_score:  5.33  frames:  87  epsilon:  0.27
episode:  658  score:  -9.16  avg_score:  4.14  frames:  159  epsilon:  0.27
episode:  659  score:  157.90  avg_score:  5.69  frames:  996  epsilon:  0.27
episode:  660  score:  18.12  avg_score:  5.78  frames:  100  epsilon:  0.27
episode:  661  score:  8.66  avg_score:  5.40  frames:  102  epsilon:  0.27
episode:  662  score:  -27.94  avg_score:  4.49  frames:  72  epsilon:  0.27
episode:  663  score:  -12.36  avg_score:  4.21  frames:  134  epsilon:  0.27
episode:  664  score:  26.98  avg_score:  4.42  frames:  88  epsilon:  0.27
episode:  665  score:  -18.49  avg_score:  3.50  frames:  105  epsilon:  0.26
episode:  666  score:  -1.98  avg_score:  3.33  frames:  201  epsilon:  0.26
episode:  667  score:  249.50  avg_score:  4.51  frames:  731  epsilon:  0.26
episode:  668  score:  -33.71  avg_score:  4.56  frames:  147  epsilon:  0.26
episode:  669  score:  0.45  avg_score:  4.81  frames:  97  epsilon:  0.26
episode:  670  score:  -12.44  avg_score:  3.46  frames:  178  epsilon:  0.26
episode:  671  score:  144.14  avg_score:  4.74  frames:  999  epsilon:  0.26
episode:  672  score:  249.05  avg_score:  7.41  frames:  473  epsilon:  0.26
episode:  673  score:  131.12  avg_score:  8.47  frames:  999  epsilon:  0.26
episode:  674  score:  -39.27  avg_score:  7.84  frames:  73  epsilon:  0.26
episode:  675  score:  58.20  avg_score:  8.58  frames:  999  epsilon:  0.26
episode:  676  score:  145.10  avg_score:  9.98  frames:  999  epsilon:  0.26
episode:  677  score:  -136.90  avg_score:  8.47  frames:  578  epsilon:  0.26
episode:  678  score:  246.35  avg_score:  12.09  frames:  328  epsilon:  0.26
episode:  679  score:  28.84  avg_score:  15.43  frames:  130  epsilon:  0.26
episode:  680  score:  122.05  avg_score:  16.63  frames:  999  epsilon:  0.26
episode:  681  score:  264.94  avg_score:  20.88  frames:  290  epsilon:  0.26
episode:  682  score:  118.72  avg_score:  22.14  frames:  999  epsilon:  0.26
episode:  683  score:  -7.50  avg_score:  22.09  frames:  71  epsilon:  0.26
episode:  684  score:  153.23  avg_score:  22.11  frames:  999  epsilon:  0.25
episode:  685  score:  -3.38  avg_score:  21.76  frames:  86  epsilon:  0.25
episode:  686  score:  255.58  avg_score:  24.17  frames:  562  epsilon:  0.25
episode:  687  score:  288.77  avg_score:  25.70  frames:  401  epsilon:  0.25
episode:  688  score:  -44.30  avg_score:  24.10  frames:  68  epsilon:  0.25
episode:  689  score:  250.50  avg_score:  29.89  frames:  465  epsilon:  0.25
episode:  690  score:  135.23  avg_score:  29.94  frames:  999  epsilon:  0.25
episode:  691  score:  93.58  avg_score:  29.71  frames:  999  epsilon:  0.25
episode:  692  score:  -41.09  avg_score:  31.05  frames:  58  epsilon:  0.25
episode:  693  score:  251.09  avg_score:  33.42  frames:  784  epsilon:  0.25
episode:  694  score:  -124.97  avg_score:  31.15  frames:  95  epsilon:  0.25
episode:  695  score:  -190.39  avg_score:  26.85  frames:  339  epsilon:  0.25
episode:  696  score:  -14.32  avg_score:  26.63  frames:  110  epsilon:  0.25
episode:  697  score:  -214.56  avg_score:  24.39  frames:  192  epsilon:  0.25
episode:  698  score:  19.54  avg_score:  24.28  frames:  999  epsilon:  0.25
episode:  699  score:  174.91  avg_score:  26.02  frames:  888  epsilon:  0.25
episode:  700  score:  142.60  avg_score:  27.45  frames:  999  epsilon:  0.25
episode:  701  score:  295.12  avg_score:  33.67  frames:  432  epsilon:  0.25
episode:  702  score:  104.29  avg_score:  36.02  frames:  999  epsilon:  0.25
episode:  703  score:  51.62  avg_score:  36.36  frames:  999  epsilon:  0.25
episode:  704  score:  262.35  avg_score:  43.43  frames:  907  epsilon:  0.24
episode:  705  score:  19.37  avg_score:  44.62  frames:  149  epsilon:  0.24
episode:  706  score:  289.94  avg_score:  45.22  frames:  868  epsilon:  0.24
episode:  707  score:  102.19  avg_score:  46.20  frames:  999  epsilon:  0.24
episode:  708  score:  -11.90  avg_score:  44.57  frames:  147  epsilon:  0.24
episode:  709  score:  111.19  avg_score:  48.61  frames:  999  epsilon:  0.24
episode:  710  score:  122.74  avg_score:  51.28  frames:  999  epsilon:  0.24
episode:  711  score:  -207.02  avg_score:  50.36  frames:  175  epsilon:  0.24
episode:  712  score:  97.26  avg_score:  51.30  frames:  999  epsilon:  0.24
episode:  713  score:  222.70  avg_score:  52.84  frames:  872  epsilon:  0.24
episode:  714  score:  -306.40  avg_score:  47.99  frames:  155  epsilon:  0.24
episode:  715  score:  187.81  avg_score:  50.47  frames:  433  epsilon:  0.24
episode:  716  score:  206.88  avg_score:  53.43  frames:  411  epsilon:  0.24
episode:  717  score:  200.36  avg_score:  55.62  frames:  923  epsilon:  0.24
episode:  718  score:  70.65  avg_score:  56.13  frames:  999  epsilon:  0.24
episode:  719  score:  280.00  avg_score:  57.75  frames:  256  epsilon:  0.24
episode:  720  score:  133.73  avg_score:  59.11  frames:  999  epsilon:  0.24
episode:  721  score:  268.01  avg_score:  65.66  frames:  879  epsilon:  0.24
episode:  722  score:  35.61  avg_score:  65.72  frames:  102  epsilon:  0.24
episode:  723  score:  131.44  avg_score:  67.59  frames:  999  epsilon:  0.24
episode:  724  score:  133.70  avg_score:  68.76  frames:  999  epsilon:  0.24
episode:  725  score:  132.06  avg_score:  68.46  frames:  999  epsilon:  0.23
episode:  726  score:  144.28  avg_score:  68.71  frames:  999  epsilon:  0.23
episode:  727  score:  270.56  avg_score:  71.94  frames:  628  epsilon:  0.23
episode:  728  score:  183.35  avg_score:  76.63  frames:  999  epsilon:  0.23
episode:  729  score:  36.53  avg_score:  75.40  frames:  999  epsilon:  0.23
episode:  730  score:  28.44  avg_score:  75.58  frames:  128  epsilon:  0.23
episode:  731  score:  -55.50  avg_score:  74.81  frames:  478  epsilon:  0.23
episode:  732  score:  145.21  avg_score:  74.70  frames:  999  epsilon:  0.23
episode:  733  score:  68.46  avg_score:  74.38  frames:  999  epsilon:  0.23
episode:  734  score:  254.20  avg_score:  76.95  frames:  852  epsilon:  0.23
episode:  735  score:  233.76  avg_score:  77.97  frames:  798  epsilon:  0.23
episode:  736  score:  27.08  avg_score:  78.17  frames:  999  epsilon:  0.23
episode:  737  score:  229.73  avg_score:  79.30  frames:  778  epsilon:  0.23
episode:  738  score:  147.08  avg_score:  84.25  frames:  999  epsilon:  0.23
episode:  739  score:  64.81  avg_score:  83.32  frames:  104  epsilon:  0.23
episode:  740  score:  35.63  avg_score:  82.99  frames:  129  epsilon:  0.23
episode:  741  score:  6.60  avg_score:  83.02  frames:  155  epsilon:  0.23
episode:  742  score:  127.29  avg_score:  83.82  frames:  999  epsilon:  0.23
episode:  743  score:  24.12  avg_score:  82.41  frames:  133  epsilon:  0.23
episode:  744  score:  34.36  avg_score:  81.28  frames:  115  epsilon:  0.23
episode:  745  score:  -5.04  avg_score:  81.24  frames:  171  epsilon:  0.23
episode:  746  score:  92.16  avg_score:  80.98  frames:  170  epsilon:  0.22
episode:  747  score:  32.48  avg_score:  78.79  frames:  128  epsilon:  0.22
episode:  748  score:  -31.53  avg_score:  79.84  frames:  92  epsilon:  0.22
episode:  749  score:  107.30  avg_score:  79.68  frames:  999  epsilon:  0.22
episode:  750  score:  2.26  avg_score:  79.97  frames:  139  epsilon:  0.22
episode:  751  score:  82.58  avg_score:  80.98  frames:  999  epsilon:  0.22
episode:  752  score:  134.01  avg_score:  81.86  frames:  999  epsilon:  0.22
episode:  753  score:  4.87  avg_score:  81.78  frames:  137  epsilon:  0.22
episode:  754  score:  252.54  avg_score:  84.11  frames:  748  epsilon:  0.22
episode:  755  score:  39.32  avg_score:  84.55  frames:  137  epsilon:  0.22
episode:  756  score:  24.01  avg_score:  84.72  frames:  110  epsilon:  0.22
episode:  757  score:  127.56  avg_score:  86.05  frames:  999  epsilon:  0.22
episode:  758  score:  97.72  avg_score:  87.12  frames:  999  epsilon:  0.22
episode:  759  score:  233.30  avg_score:  87.87  frames:  346  epsilon:  0.22
episode:  760  score:  256.68  avg_score:  90.25  frames:  807  epsilon:  0.22
episode:  761  score:  -196.17  avg_score:  88.21  frames:  211  epsilon:  0.22
episode:  762  score:  -263.92  avg_score:  85.85  frames:  366  epsilon:  0.22
episode:  763  score:  234.13  avg_score:  88.31  frames:  578  epsilon:  0.22
episode:  764  score:  236.50  avg_score:  90.41  frames:  574  epsilon:  0.22
episode:  765  score:  271.16  avg_score:  93.30  frames:  456  epsilon:  0.22
episode:  766  score:  265.14  avg_score:  95.97  frames:  633  epsilon:  0.22
episode:  767  score:  148.32  avg_score:  94.96  frames:  999  epsilon:  0.22
episode:  768  score:  168.73  avg_score:  96.99  frames:  999  epsilon:  0.22
episode:  769  score:  -27.50  avg_score:  96.71  frames:  141  epsilon:  0.21
episode:  770  score:  148.39  avg_score:  98.32  frames:  999  epsilon:  0.21
episode:  771  score:  28.75  avg_score:  97.16  frames:  103  epsilon:  0.21
episode:  772  score:  129.14  avg_score:  95.96  frames:  999  epsilon:  0.21
episode:  773  score:  122.18  avg_score:  95.87  frames:  999  epsilon:  0.21
episode:  774  score:  221.50  avg_score:  98.48  frames:  539  epsilon:  0.21
episode:  775  score:  278.02  avg_score:  100.68  frames:  200  epsilon:  0.21
episode:  776  score:  137.33  avg_score:  100.60  frames:  999  epsilon:  0.21
episode:  777  score:  260.46  avg_score:  104.58  frames:  404  epsilon:  0.21
episode:  778  score:  248.22  avg_score:  104.59  frames:  227  epsilon:  0.21
episode:  779  score:  284.11  avg_score:  107.15  frames:  438  epsilon:  0.21
episode:  780  score:  268.60  avg_score:  108.61  frames:  308  epsilon:  0.21
episode:  781  score:  33.64  avg_score:  106.30  frames:  148  epsilon:  0.21
episode:  782  score:  262.20  avg_score:  107.73  frames:  690  epsilon:  0.21
episode:  783  score:  243.36  avg_score:  110.24  frames:  579  epsilon:  0.21
episode:  784  score:  -11.38  avg_score:  108.60  frames:  140  epsilon:  0.21
episode:  785  score:  28.85  avg_score:  108.92  frames:  127  epsilon:  0.21
episode:  786  score:  250.78  avg_score:  108.87  frames:  416  epsilon:  0.21
episode:  787  score:  222.31  avg_score:  108.21  frames:  489  epsilon:  0.21
episode:  788  score:  260.32  avg_score:  111.25  frames:  491  epsilon:  0.21
episode:  789  score:  221.03  avg_score:  110.96  frames:  451  epsilon:  0.21
episode:  790  score:  201.56  avg_score:  111.62  frames:  612  epsilon:  0.21
episode:  791  score:  16.58  avg_score:  110.85  frames:  133  epsilon:  0.21
episode:  792  score:  24.52  avg_score:  111.51  frames:  163  epsilon:  0.21
episode:  793  score:  141.95  avg_score:  110.42  frames:  999  epsilon:  0.20
episode:  794  score:  13.85  avg_score:  111.80  frames:  999  epsilon:  0.20
episode:  795  score:  290.75  avg_score:  116.61  frames:  479  epsilon:  0.20
episode:  796  score:  114.95  avg_score:  117.91  frames:  999  epsilon:  0.20
episode:  797  score:  35.04  avg_score:  120.40  frames:  147  epsilon:  0.20
episode:  798  score:  220.45  avg_score:  122.41  frames:  415  epsilon:  0.20
episode:  799  score:  -171.80  avg_score:  118.95  frames:  89  epsilon:  0.20
episode:  800  score:  263.63  avg_score:  120.16  frames:  688  epsilon:  0.20
episode:  801  score:  123.14  avg_score:  118.44  frames:  999  epsilon:  0.20
episode:  802  score:  -56.63  avg_score:  116.83  frames:  62  epsilon:  0.20
episode:  803  score:  -8.17  avg_score:  116.23  frames:  91  epsilon:  0.20
episode:  804  score:  107.57  avg_score:  114.68  frames:  999  epsilon:  0.20
episode:  805  score:  144.78  avg_score:  115.94  frames:  999  epsilon:  0.20
episode:  806  score:  211.92  avg_score:  115.16  frames:  759  epsilon:  0.20
episode:  807  score:  159.88  avg_score:  115.73  frames:  577  epsilon:  0.20
episode:  808  score:  -22.81  avg_score:  115.62  frames:  90  epsilon:  0.20
episode:  809  score:  92.10  avg_score:  115.43  frames:  999  epsilon:  0.20
episode:  810  score:  231.30  avg_score:  116.52  frames:  984  epsilon:  0.20
episode:  811  score:  110.56  avg_score:  119.69  frames:  999  epsilon:  0.20
episode:  812  score:  239.98  avg_score:  121.12  frames:  486  epsilon:  0.20
episode:  813  score:  210.85  avg_score:  121.00  frames:  877  epsilon:  0.20
episode:  814  score:  9.06  avg_score:  124.16  frames:  111  epsilon:  0.20
episode:  815  score:  290.91  avg_score:  125.19  frames:  632  epsilon:  0.20
episode:  816  score:  254.02  avg_score:  125.66  frames:  336  epsilon:  0.20
episode:  817  score:  270.96  avg_score:  126.37  frames:  257  epsilon:  0.20
episode:  818  score:  7.08  avg_score:  125.73  frames:  94  epsilon:  0.19
episode:  819  score:  209.94  avg_score:  125.03  frames:  856  epsilon:  0.19
episode:  820  score:  123.83  avg_score:  124.93  frames:  999  epsilon:  0.19
episode:  821  score:  236.73  avg_score:  124.62  frames:  403  epsilon:  0.19
episode:  822  score:  222.90  avg_score:  126.49  frames:  626  epsilon:  0.19
episode:  823  score:  256.17  avg_score:  127.74  frames:  203  epsilon:  0.19
episode:  824  score:  233.63  avg_score:  128.74  frames:  842  epsilon:  0.19
episode:  825  score:  0.42  avg_score:  127.42  frames:  80  epsilon:  0.19
episode:  826  score:  -157.36  avg_score:  124.40  frames:  269  epsilon:  0.19
episode:  827  score:  -5.81  avg_score:  121.64  frames:  136  epsilon:  0.19
episode:  828  score:  94.09  avg_score:  120.75  frames:  999  epsilon:  0.19
episode:  829  score:  218.87  avg_score:  122.57  frames:  546  epsilon:  0.19
episode:  830  score:  244.47  avg_score:  124.73  frames:  590  epsilon:  0.19
episode:  831  score:  267.67  avg_score:  127.96  frames:  400  epsilon:  0.19
episode:  832  score:  234.47  avg_score:  128.86  frames:  337  epsilon:  0.19
episode:  833  score:  277.52  avg_score:  130.95  frames:  430  epsilon:  0.19
episode:  834  score:  0.40  avg_score:  128.41  frames:  201  epsilon:  0.19
episode:  835  score:  266.03  avg_score:  128.73  frames:  360  epsilon:  0.19
episode:  836  score:  267.56  avg_score:  131.14  frames:  380  epsilon:  0.19
episode:  837  score:  160.71  avg_score:  130.45  frames:  721  epsilon:  0.19
episode:  838  score:  280.64  avg_score:  131.78  frames:  342  epsilon:  0.19
episode:  839  score:  247.97  avg_score:  133.61  frames:  291  epsilon:  0.19
episode:  840  score:  212.69  avg_score:  135.38  frames:  373  epsilon:  0.19
episode:  841  score:  282.27  avg_score:  138.14  frames:  312  epsilon:  0.19
episode:  842  score:  244.07  avg_score:  139.31  frames:  247  epsilon:  0.19
episode:  843  score:  239.63  avg_score:  141.46  frames:  698  epsilon:  0.19
episode:  844  score:  233.63  avg_score:  143.46  frames:  623  epsilon:  0.18
episode:  845  score:  226.47  avg_score:  145.77  frames:  627  epsilon:  0.18
episode:  846  score:  271.21  avg_score:  147.56  frames:  365  epsilon:  0.18
episode:  847  score:  159.71  avg_score:  148.83  frames:  999  epsilon:  0.18
episode:  848  score:  263.39  avg_score:  151.78  frames:  751  epsilon:  0.18
episode:  849  score:  -0.43  avg_score:  150.71  frames:  105  epsilon:  0.18
episode:  850  score:  201.88  avg_score:  152.70  frames:  970  epsilon:  0.18
episode:  851  score:  215.61  avg_score:  154.03  frames:  407  epsilon:  0.18
episode:  852  score:  228.41  avg_score:  154.98  frames:  500  epsilon:  0.18
episode:  853  score:  -14.09  avg_score:  154.79  frames:  171  epsilon:  0.18
episode:  854  score:  -77.62  avg_score:  151.48  frames:  122  epsilon:  0.18
episode:  855  score:  256.82  avg_score:  153.66  frames:  605  epsilon:  0.18
episode:  856  score:  -24.67  avg_score:  153.17  frames:  150  epsilon:  0.18
episode:  857  score:  192.17  avg_score:  153.82  frames:  319  epsilon:  0.18
episode:  858  score:  242.04  avg_score:  155.26  frames:  361  epsilon:  0.18
episode:  859  score:  237.98  avg_score:  155.31  frames:  288  epsilon:  0.18
episode:  860  score:  -25.79  avg_score:  152.48  frames:  999  epsilon:  0.18
episode:  861  score:  24.01  avg_score:  154.69  frames:  91  epsilon:  0.18
episode:  862  score:  -76.55  avg_score:  156.56  frames:  315  epsilon:  0.18
episode:  863  score:  242.75  avg_score:  156.65  frames:  602  epsilon:  0.18
episode:  864  score:  -17.94  avg_score:  154.10  frames:  111  epsilon:  0.18
episode:  865  score:  -9.72  avg_score:  151.29  frames:  103  epsilon:  0.18
episode:  866  score:  268.91  avg_score:  151.33  frames:  179  epsilon:  0.18
episode:  867  score:  175.60  avg_score:  151.60  frames:  370  epsilon:  0.18
episode:  868  score:  61.16  avg_score:  150.53  frames:  94  epsilon:  0.18
episode:  869  score:  37.21  avg_score:  151.17  frames:  137  epsilon:  0.18
episode:  870  score:  120.17  avg_score:  150.89  frames:  999  epsilon:  0.18
episode:  871  score:  244.09  avg_score:  153.05  frames:  307  epsilon:  0.18
episode:  872  score:  274.11  avg_score:  154.50  frames:  778  epsilon:  0.17
episode:  873  score:  -40.14  avg_score:  152.87  frames:  83  epsilon:  0.17
episode:  874  score:  -122.01  avg_score:  149.44  frames:  147  epsilon:  0.17
episode:  875  score:  -29.25  avg_score:  146.36  frames:  97  epsilon:  0.17
episode:  876  score:  -15.44  avg_score:  144.84  frames:  98  epsilon:  0.17
episode:  877  score:  18.79  avg_score:  142.42  frames:  159  epsilon:  0.17
episode:  878  score:  34.66  avg_score:  140.28  frames:  109  epsilon:  0.17
episode:  879  score:  273.01  avg_score:  140.17  frames:  412  epsilon:  0.17
episode:  880  score:  249.30  avg_score:  139.98  frames:  977  epsilon:  0.17
episode:  881  score:  -100.21  avg_score:  138.64  frames:  123  epsilon:  0.17
episode:  882  score:  267.92  avg_score:  138.70  frames:  917  epsilon:  0.17
episode:  883  score:  35.80  avg_score:  136.62  frames:  999  epsilon:  0.17
episode:  884  score:  -29.24  avg_score:  136.44  frames:  90  epsilon:  0.17
episode:  885  score:  269.14  avg_score:  138.85  frames:  435  epsilon:  0.17
episode:  886  score:  121.27  avg_score:  137.55  frames:  999  epsilon:  0.17
episode:  887  score:  5.99  avg_score:  135.39  frames:  125  epsilon:  0.17
episode:  888  score:  -113.36  avg_score:  131.65  frames:  134  epsilon:  0.17
episode:  889  score:  149.44  avg_score:  130.94  frames:  999  epsilon:  0.17
episode:  890  score:  -37.79  avg_score:  128.54  frames:  101  epsilon:  0.17
episode:  891  score:  26.13  avg_score:  128.64  frames:  139  epsilon:  0.17
episode:  892  score:  234.50  avg_score:  130.74  frames:  351  epsilon:  0.17
episode:  893  score:  221.48  avg_score:  131.53  frames:  815  epsilon:  0.17
episode:  894  score:  225.23  avg_score:  133.65  frames:  799  epsilon:  0.17
episode:  895  score:  223.26  avg_score:  132.97  frames:  228  epsilon:  0.17
episode:  896  score:  12.76  avg_score:  131.95  frames:  116  epsilon:  0.17
episode:  897  score:  269.28  avg_score:  134.29  frames:  227  epsilon:  0.17
episode:  898  score:  40.36  avg_score:  132.49  frames:  151  epsilon:  0.17
episode:  899  score:  25.42  avg_score:  134.46  frames:  173  epsilon:  0.17
episode:  900  score:  260.04  avg_score:  134.43  frames:  795  epsilon:  0.17
episode:  901  score:  261.69  avg_score:  135.81  frames:  658  epsilon:  0.17
episode:  902  score:  233.57  avg_score:  138.72  frames:  645  epsilon:  0.16
episode:  903  score:  7.59  avg_score:  138.87  frames:  226  epsilon:  0.16
episode:  904  score:  275.77  avg_score:  140.56  frames:  293  epsilon:  0.16
episode:  905  score:  -11.00  avg_score:  139.00  frames:  120  epsilon:  0.16
episode:  906  score:  193.96  avg_score:  138.82  frames:  464  epsilon:  0.16
episode:  907  score:  134.82  avg_score:  138.57  frames:  999  epsilon:  0.16
episode:  908  score:  -11.53  avg_score:  138.68  frames:  999  epsilon:  0.16
episode:  909  score:  -424.71  avg_score:  133.51  frames:  323  epsilon:  0.16
episode:  910  score:  266.42  avg_score:  133.86  frames:  340  epsilon:  0.16
episode:  911  score:  292.47  avg_score:  135.68  frames:  221  epsilon:  0.16
episode:  912  score:  243.74  avg_score:  135.72  frames:  248  epsilon:  0.16
episode:  913  score:  243.37  avg_score:  136.05  frames:  573  epsilon:  0.16
episode:  914  score:  285.41  avg_score:  138.81  frames:  582  epsilon:  0.16
episode:  915  score:  -105.02  avg_score:  134.85  frames:  131  epsilon:  0.16
episode:  916  score:  237.89  avg_score:  134.69  frames:  316  epsilon:  0.16
episode:  917  score:  223.87  avg_score:  134.22  frames:  304  epsilon:  0.16
episode:  918  score:  180.67  avg_score:  135.95  frames:  667  epsilon:  0.16
episode:  919  score:  279.44  avg_score:  136.65  frames:  343  epsilon:  0.16
episode:  920  score:  263.75  avg_score:  138.05  frames:  420  epsilon:  0.16
episode:  921  score:  215.38  avg_score:  137.83  frames:  276  epsilon:  0.16
episode:  922  score:  239.26  avg_score:  138.00  frames:  718  epsilon:  0.16
episode:  923  score:  183.38  avg_score:  137.27  frames:  614  epsilon:  0.16
episode:  924  score:  118.17  avg_score:  136.12  frames:  999  epsilon:  0.16
episode:  925  score:  241.80  avg_score:  138.53  frames:  473  epsilon:  0.16
episode:  926  score:  147.24  avg_score:  141.57  frames:  999  epsilon:  0.16
episode:  927  score:  237.02  avg_score:  144.00  frames:  447  epsilon:  0.16
episode:  928  score:  299.82  avg_score:  146.06  frames:  234  epsilon:  0.16
episode:  929  score:  -174.63  avg_score:  142.13  frames:  535  epsilon:  0.16
episode:  930  score:  -0.44  avg_score:  139.68  frames:  97  epsilon:  0.16
episode:  931  score:  -14.83  avg_score:  136.85  frames:  92  epsilon:  0.16
episode:  932  score:  -42.09  avg_score:  134.09  frames:  128  epsilon:  0.16
episode:  933  score:  -96.61  avg_score:  130.34  frames:  135  epsilon:  0.15
episode:  934  score:  293.19  avg_score:  133.27  frames:  244  epsilon:  0.15
episode:  935  score:  262.42  avg_score:  133.24  frames:  267  epsilon:  0.15
episode:  936  score:  140.72  avg_score:  131.97  frames:  913  epsilon:  0.15
episode:  937  score:  -25.50  avg_score:  130.11  frames:  82  epsilon:  0.15
episode:  938  score:  -220.28  avg_score:  125.10  frames:  173  epsilon:  0.15
episode:  939  score:  -80.69  avg_score:  121.81  frames:  157  epsilon:  0.15
episode:  940  score:  16.64  avg_score:  119.85  frames:  88  epsilon:  0.15
episode:  941  score:  229.25  avg_score:  119.32  frames:  615  epsilon:  0.15
episode:  942  score:  -23.67  avg_score:  116.64  frames:  65  epsilon:  0.15
episode:  943  score:  -15.67  avg_score:  114.09  frames:  107  epsilon:  0.15
episode:  944  score:  -17.43  avg_score:  111.58  frames:  100  epsilon:  0.15
episode:  945  score:  203.99  avg_score:  111.35  frames:  642  epsilon:  0.15
episode:  946  score:  -51.69  avg_score:  108.12  frames:  56  epsilon:  0.15
episode:  947  score:  -46.22  avg_score:  106.07  frames:  95  epsilon:  0.15
episode:  948  score:  -86.26  avg_score:  102.57  frames:  88  epsilon:  0.15
episode:  949  score:  218.19  avg_score:  104.75  frames:  214  epsilon:  0.15
episode:  950  score:  97.10  avg_score:  103.71  frames:  999  epsilon:  0.15
episode:  951  score:  238.01  avg_score:  103.93  frames:  282  epsilon:  0.15
episode:  952  score:  6.01  avg_score:  101.71  frames:  103  epsilon:  0.15
episode:  953  score:  158.26  avg_score:  103.43  frames:  449  epsilon:  0.15
episode:  954  score:  -18.85  avg_score:  104.02  frames:  94  epsilon:  0.15
episode:  955  score:  300.47  avg_score:  104.45  frames:  202  epsilon:  0.15
episode:  956  score:  45.27  avg_score:  105.15  frames:  99  epsilon:  0.15
episode:  957  score:  -7.63  avg_score:  103.16  frames:  76  epsilon:  0.15
episode:  958  score:  18.59  avg_score:  100.92  frames:  132  epsilon:  0.15
episode:  959  score:  241.85  avg_score:  100.96  frames:  747  epsilon:  0.15
episode:  960  score:  -72.60  avg_score:  100.49  frames:  85  epsilon:  0.15
episode:  961  score:  19.15  avg_score:  100.44  frames:  247  epsilon:  0.15
episode:  962  score:  -111.51  avg_score:  100.09  frames:  112  epsilon:  0.15
episode:  963  score:  -2.43  avg_score:  97.64  frames:  141  epsilon:  0.15
episode:  964  score:  24.12  avg_score:  98.06  frames:  141  epsilon:  0.15
episode:  965  score:  -53.57  avg_score:  97.62  frames:  202  epsilon:  0.15
episode:  966  score:  -7.76  avg_score:  94.86  frames:  999  epsilon:  0.14
episode:  967  score:  1.29  avg_score:  93.11  frames:  142  epsilon:  0.14
episode:  968  score:  -0.45  avg_score:  92.50  frames:  111  epsilon:  0.14
episode:  969  score:  68.86  avg_score:  92.81  frames:  999  epsilon:  0.14
episode:  970  score:  9.77  avg_score:  91.71  frames:  153  epsilon:  0.14
episode:  971  score:  4.33  avg_score:  89.31  frames:  149  epsilon:  0.14
episode:  972  score:  19.36  avg_score:  86.77  frames:  192  epsilon:  0.14
episode:  973  score:  34.15  avg_score:  87.51  frames:  999  epsilon:  0.14
episode:  974  score:  18.19  avg_score:  88.91  frames:  999  epsilon:  0.14
episode:  975  score:  40.11  avg_score:  89.60  frames:  999  epsilon:  0.14
episode:  976  score:  62.88  avg_score:  90.39  frames:  999  epsilon:  0.14
episode:  977  score:  275.67  avg_score:  92.96  frames:  225  epsilon:  0.14
episode:  978  score:  224.89  avg_score:  94.86  frames:  360  epsilon:  0.14
episode:  979  score:  240.99  avg_score:  94.54  frames:  520  epsilon:  0.14
episode:  980  score:  -7.30  avg_score:  91.97  frames:  999  epsilon:  0.14
episode:  981  score:  209.98  avg_score:  95.07  frames:  383  epsilon:  0.14
episode:  982  score:  300.69  avg_score:  95.40  frames:  367  epsilon:  0.14
episode:  983  score:  269.19  avg_score:  97.74  frames:  226  epsilon:  0.14
episode:  984  score:  278.69  avg_score:  100.82  frames:  578  epsilon:  0.14
episode:  985  score:  -505.00  avg_score:  93.07  frames:  992  epsilon:  0.14
episode:  986  score:  0.74  avg_score:  91.87  frames:  999  epsilon:  0.14
episode:  987  score:  245.60  avg_score:  94.26  frames:  331  epsilon:  0.14
episode:  988  score:  174.10  avg_score:  97.14  frames:  870  epsilon:  0.14
episode:  989  score:  253.87  avg_score:  98.18  frames:  234  epsilon:  0.14
episode:  990  score:  -54.42  avg_score:  98.02  frames:  999  epsilon:  0.14
episode:  991  score:  227.35  avg_score:  100.03  frames:  654  epsilon:  0.14
episode:  992  score:  175.23  avg_score:  99.44  frames:  871  epsilon:  0.14
episode:  993  score:  249.18  avg_score:  99.71  frames:  559  epsilon:  0.14
episode:  994  score:  162.61  avg_score:  99.09  frames:  747  epsilon:  0.14
episode:  995  score:  236.52  avg_score:  99.22  frames:  246  epsilon:  0.14
episode:  996  score:  231.68  avg_score:  101.41  frames:  719  epsilon:  0.14
episode:  997  score:  43.78  avg_score:  99.15  frames:  999  epsilon:  0.14
episode:  998  score:  240.51  avg_score:  101.16  frames:  371  epsilon:  0.14
episode:  999  score:  223.97  avg_score:  103.14  frames:  446  epsilon:  0.14
episode:  1000  score:  262.28  avg_score:  103.16  frames:  295  epsilon:  0.14
episode:  1001  score:  -56.26  avg_score:  99.98  frames:  999  epsilon:  0.14
episode:  1002  score:  -171.17  avg_score:  95.94  frames:  310  epsilon:  0.13
episode:  1003  score:  307.81  avg_score:  98.94  frames:  335  epsilon:  0.13
episode:  1004  score:  196.25  avg_score:  98.14  frames:  644  epsilon:  0.13
episode:  1005  score:  303.93  avg_score:  101.29  frames:  345  epsilon:  0.13
episode:  1006  score:  210.30  avg_score:  101.46  frames:  300  epsilon:  0.13
episode:  1007  score:  220.25  avg_score:  102.31  frames:  429  epsilon:  0.13
episode:  1008  score:  50.89  avg_score:  102.93  frames:  999  epsilon:  0.13
episode:  1009  score:  -23.10  avg_score:  106.95  frames:  119  epsilon:  0.13
episode:  1010  score:  -161.51  avg_score:  102.67  frames:  165  epsilon:  0.13
episode:  1011  score:  291.40  avg_score:  102.66  frames:  953  epsilon:  0.13
episode:  1012  score:  145.07  avg_score:  101.67  frames:  497  epsilon:  0.13
episode:  1013  score:  -194.68  avg_score:  97.29  frames:  58  epsilon:  0.13
episode:  1014  score:  157.61  avg_score:  96.02  frames:  719  epsilon:  0.13
episode:  1015  score:  204.84  avg_score:  99.11  frames:  633  epsilon:  0.13
episode:  1016  score:  250.97  avg_score:  99.25  frames:  243  epsilon:  0.13
episode:  1017  score:  269.99  avg_score:  99.71  frames:  390  epsilon:  0.13
episode:  1018  score:  278.02  avg_score:  100.68  frames:  462  epsilon:  0.13
episode:  1019  score:  256.20  avg_score:  100.45  frames:  443  epsilon:  0.13
episode:  1020  score:  190.82  avg_score:  99.72  frames:  425  epsilon:  0.13
episode:  1021  score:  247.32  avg_score:  100.04  frames:  196  epsilon:  0.13
episode:  1022  score:  243.92  avg_score:  100.08  frames:  334  epsilon:  0.13
episode:  1023  score:  254.57  avg_score:  100.80  frames:  442  epsilon:  0.13
episode:  1024  score:  299.81  avg_score:  102.61  frames:  299  epsilon:  0.13
episode:  1025  score:  246.80  avg_score:  102.66  frames:  262  epsilon:  0.13
episode:  1026  score:  261.02  avg_score:  103.80  frames:  267  epsilon:  0.13
episode:  1027  score:  269.80  avg_score:  104.13  frames:  433  epsilon:  0.13
episode:  1028  score:  243.38  avg_score:  103.56  frames:  311  epsilon:  0.13
episode:  1029  score:  -235.86  avg_score:  102.95  frames:  803  epsilon:  0.13
episode:  1030  score:  260.90  avg_score:  105.57  frames:  514  epsilon:  0.13
episode:  1031  score:  257.30  avg_score:  108.29  frames:  384  epsilon:  0.13
episode:  1032  score:  -228.85  avg_score:  106.42  frames:  161  epsilon:  0.13
episode:  1033  score:  286.35  avg_score:  110.25  frames:  623  epsilon:  0.13
episode:  1034  score:  -20.48  avg_score:  107.11  frames:  71  epsilon:  0.13
episode:  1035  score:  -36.03  avg_score:  104.13  frames:  114  epsilon:  0.13
episode:  1036  score:  -16.88  avg_score:  102.55  frames:  88  epsilon:  0.13
episode:  1037  score:  268.04  avg_score:  105.49  frames:  190  epsilon:  0.13
episode:  1038  score:  -377.98  avg_score:  103.91  frames:  403  epsilon:  0.13
episode:  1039  score:  -153.08  avg_score:  103.19  frames:  132  epsilon:  0.13
episode:  1040  score:  232.08  avg_score:  105.34  frames:  238  epsilon:  0.13
episode:  1041  score:  -485.37  avg_score:  98.19  frames:  91  epsilon:  0.12
episode:  1042  score:  230.93  avg_score:  100.74  frames:  405  epsilon:  0.12
episode:  1043  score:  264.97  avg_score:  103.55  frames:  598  epsilon:  0.12
episode:  1044  score:  -50.85  avg_score:  103.21  frames:  398  epsilon:  0.12
episode:  1045  score:  257.68  avg_score:  103.75  frames:  247  epsilon:  0.12
episode:  1046  score:  228.65  avg_score:  106.55  frames:  494  epsilon:  0.12
episode:  1047  score:  259.94  avg_score:  109.61  frames:  310  epsilon:  0.12
episode:  1048  score:  251.92  avg_score:  113.00  frames:  382  epsilon:  0.12
episode:  1049  score:  221.95  avg_score:  113.03  frames:  625  epsilon:  0.12
episode:  1050  score:  265.34  avg_score:  114.72  frames:  220  epsilon:  0.12
episode:  1051  score:  239.11  avg_score:  114.73  frames:  383  epsilon:  0.12
episode:  1052  score:  291.44  avg_score:  117.58  frames:  561  epsilon:  0.12
episode:  1053  score:  203.27  avg_score:  118.03  frames:  524  epsilon:  0.12
episode:  1054  score:  -300.80  avg_score:  115.21  frames:  428  epsilon:  0.12
episode:  1055  score:  250.71  avg_score:  114.71  frames:  195  epsilon:  0.12
episode:  1056  score:  287.08  avg_score:  117.13  frames:  181  epsilon:  0.12
episode:  1057  score:  266.38  avg_score:  119.87  frames:  582  epsilon:  0.12
episode:  1058  score:  245.80  avg_score:  122.14  frames:  236  epsilon:  0.12
episode:  1059  score:  199.87  avg_score:  121.72  frames:  212  epsilon:  0.12
episode:  1060  score:  255.88  avg_score:  125.01  frames:  247  epsilon:  0.12
episode:  1061  score:  198.16  avg_score:  126.80  frames:  486  epsilon:  0.12
episode:  1062  score:  250.37  avg_score:  130.42  frames:  311  epsilon:  0.12
episode:  1063  score:  243.51  avg_score:  132.88  frames:  292  epsilon:  0.12
episode:  1064  score:  234.51  avg_score:  134.98  frames:  512  epsilon:  0.12
episode:  1065  score:  231.17  avg_score:  137.83  frames:  587  epsilon:  0.12
episode:  1066  score:  241.75  avg_score:  140.32  frames:  648  epsilon:  0.12
episode:  1067  score:  229.65  avg_score:  142.61  frames:  472  epsilon:  0.12
episode:  1068  score:  243.71  avg_score:  145.05  frames:  222  epsilon:  0.12
episode:  1069  score:  263.61  avg_score:  147.00  frames:  178  epsilon:  0.12
episode:  1070  score:  267.69  avg_score:  149.58  frames:  265  epsilon:  0.12
episode:  1071  score:  283.35  avg_score:  152.37  frames:  155  epsilon:  0.12
episode:  1072  score:  264.74  avg_score:  154.82  frames:  252  epsilon:  0.12
episode:  1073  score:  -27.41  avg_score:  154.20  frames:  99  epsilon:  0.12
episode:  1074  score:  156.89  avg_score:  155.59  frames:  350  epsilon:  0.12
episode:  1075  score:  25.84  avg_score:  155.45  frames:  109  epsilon:  0.12
episode:  1076  score:  16.58  avg_score:  154.99  frames:  107  epsilon:  0.12
episode:  1077  score:  247.27  avg_score:  154.70  frames:  249  epsilon:  0.12
episode:  1078  score:  33.09  avg_score:  152.78  frames:  104  epsilon:  0.12
episode:  1079  score:  306.87  avg_score:  153.44  frames:  378  epsilon:  0.12
episode:  1080  score:  276.11  avg_score:  156.28  frames:  270  epsilon:  0.12
episode:  1081  score:  268.44  avg_score:  156.86  frames:  387  epsilon:  0.12
episode:  1082  score:  -32.91  avg_score:  153.53  frames:  204  epsilon:  0.11
episode:  1083  score:  245.96  avg_score:  153.29  frames:  240  epsilon:  0.11
episode:  1084  score:  298.06  avg_score:  153.49  frames:  262  epsilon:  0.11
episode:  1085  score:  234.16  avg_score:  160.88  frames:  297  epsilon:  0.11
episode:  1086  score:  225.84  avg_score:  163.13  frames:  832  epsilon:  0.11
episode:  1087  score:  245.96  avg_score:  163.13  frames:  278  epsilon:  0.11
episode:  1088  score:  52.85  avg_score:  161.92  frames:  126  epsilon:  0.11
episode:  1089  score:  -123.82  avg_score:  158.14  frames:  66  epsilon:  0.11
episode:  1090  score:  260.95  avg_score:  161.30  frames:  244  epsilon:  0.11
episode:  1091  score:  24.33  avg_score:  159.27  frames:  195  epsilon:  0.11
episode:  1092  score:  233.94  avg_score:  159.85  frames:  350  epsilon:  0.11
episode:  1093  score:  254.32  avg_score:  159.91  frames:  320  epsilon:  0.11
episode:  1094  score:  287.46  avg_score:  161.15  frames:  328  epsilon:  0.11
episode:  1095  score:  47.49  avg_score:  159.26  frames:  134  epsilon:  0.11
episode:  1096  score:  252.53  avg_score:  159.47  frames:  312  epsilon:  0.11
episode:  1097  score:  239.23  avg_score:  161.43  frames:  373  epsilon:  0.11
episode:  1098  score:  254.63  avg_score:  161.57  frames:  415  epsilon:  0.11
episode:  1099  score:  266.03  avg_score:  161.99  frames:  310  epsilon:  0.11
episode:  1100  score:  276.78  avg_score:  162.13  frames:  395  epsilon:  0.11
episode:  1101  score:  -317.59  avg_score:  159.52  frames:  337  epsilon:  0.11
episode:  1102  score:  -325.78  avg_score:  157.97  frames:  202  epsilon:  0.11
episode:  1103  score:  249.00  avg_score:  157.39  frames:  465  epsilon:  0.11
episode:  1104  score:  210.08  avg_score:  157.53  frames:  571  epsilon:  0.11
episode:  1105  score:  293.25  avg_score:  157.42  frames:  421  epsilon:  0.11
episode:  1106  score:  -66.19  avg_score:  154.65  frames:  999  epsilon:  0.11
episode:  1107  score:  219.29  avg_score:  154.64  frames:  325  epsilon:  0.11
episode:  1108  score:  270.81  avg_score:  156.84  frames:  309  epsilon:  0.11
episode:  1109  score:  266.35  avg_score:  159.74  frames:  275  epsilon:  0.11
episode:  1110  score:  274.14  avg_score:  164.09  frames:  306  epsilon:  0.11
episode:  1111  score:  275.11  avg_score:  163.93  frames:  628  epsilon:  0.11
episode:  1112  score:  289.18  avg_score:  165.37  frames:  352  epsilon:  0.11
episode:  1113  score:  289.11  avg_score:  170.21  frames:  442  epsilon:  0.11
episode:  1114  score:  294.74  avg_score:  171.58  frames:  311  epsilon:  0.11
episode:  1115  score:  270.42  avg_score:  172.24  frames:  258  epsilon:  0.11
episode:  1116  score:  308.46  avg_score:  172.81  frames:  387  epsilon:  0.11
episode:  1117  score:  253.62  avg_score:  172.65  frames:  294  epsilon:  0.11
episode:  1118  score:  275.43  avg_score:  172.62  frames:  195  epsilon:  0.11
episode:  1119  score:  158.07  avg_score:  171.64  frames:  999  epsilon:  0.11
episode:  1120  score:  260.17  avg_score:  172.33  frames:  580  epsilon:  0.11
episode:  1121  score:  255.26  avg_score:  172.41  frames:  593  epsilon:  0.11
episode:  1122  score:  211.37  avg_score:  172.09  frames:  525  epsilon:  0.11
episode:  1123  score:  38.68  avg_score:  169.93  frames:  111  epsilon:  0.11
episode:  1124  score:  8.80  avg_score:  167.02  frames:  128  epsilon:  0.11
episode:  1125  score:  272.89  avg_score:  167.28  frames:  583  epsilon:  0.11
episode:  1126  score:  -206.55  avg_score:  162.60  frames:  121  epsilon:  0.11
episode:  1127  score:  258.31  avg_score:  162.49  frames:  380  epsilon:  0.11
episode:  1128  score:  261.62  avg_score:  162.67  frames:  570  epsilon:  0.10
episode:  1129  score:  263.51  avg_score:  167.67  frames:  279  epsilon:  0.10
episode:  1130  score:  261.34  avg_score:  167.67  frames:  305  epsilon:  0.10
episode:  1131  score:  -132.09  avg_score:  163.78  frames:  64  epsilon:  0.10
episode:  1132  score:  -107.90  avg_score:  164.99  frames:  352  epsilon:  0.10
episode:  1133  score:  282.23  avg_score:  164.94  frames:  355  epsilon:  0.10
episode:  1134  score:  305.63  avg_score:  168.21  frames:  345  epsilon:  0.10
episode:  1135  score:  300.39  avg_score:  171.57  frames:  292  epsilon:  0.10
episode:  1136  score:  286.58  avg_score:  174.60  frames:  283  epsilon:  0.10
episode:  1137  score:  267.63  avg_score:  174.60  frames:  301  epsilon:  0.10
episode:  1138  score:  22.77  avg_score:  178.61  frames:  133  epsilon:  0.10
episode:  1139  score:  234.59  avg_score:  182.48  frames:  254  epsilon:  0.10
episode:  1140  score:  258.61  avg_score:  182.75  frames:  376  epsilon:  0.10
episode:  1141  score:  32.15  avg_score:  187.92  frames:  164  epsilon:  0.10
episode:  1142  score:  262.35  avg_score:  188.24  frames:  446  epsilon:  0.10
episode:  1143  score:  253.12  avg_score:  188.12  frames:  329  epsilon:  0.10
episode:  1144  score:  300.22  avg_score:  191.63  frames:  250  epsilon:  0.10
episode:  1145  score:  247.96  avg_score:  191.53  frames:  542  epsilon:  0.10
episode:  1146  score:  -0.00  avg_score:  189.25  frames:  144  epsilon:  0.10
episode:  1147  score:  264.74  avg_score:  189.29  frames:  325  epsilon:  0.10
episode:  1148  score:  282.35  avg_score:  189.60  frames:  679  epsilon:  0.10
episode:  1149  score:  261.22  avg_score:  189.99  frames:  356  epsilon:  0.10
episode:  1150  score:  285.27  avg_score:  190.19  frames:  384  epsilon:  0.10
episode:  1151  score:  6.01  avg_score:  187.86  frames:  105  epsilon:  0.10
episode:  1152  score:  236.09  avg_score:  187.31  frames:  317  epsilon:  0.10
episode:  1153  score:  247.40  avg_score:  187.75  frames:  281  epsilon:  0.10
episode:  1154  score:  280.78  avg_score:  193.56  frames:  415  epsilon:  0.10
episode:  1155  score:  -14.71  avg_score:  190.91  frames:  108  epsilon:  0.10
episode:  1156  score:  262.48  avg_score:  190.66  frames:  204  epsilon:  0.10
episode:  1157  score:  262.43  avg_score:  190.62  frames:  516  epsilon:  0.10
episode:  1158  score:  266.05  avg_score:  190.83  frames:  225  epsilon:  0.10
episode:  1159  score:  263.95  avg_score:  191.47  frames:  236  epsilon:  0.10
episode:  1160  score:  267.41  avg_score:  191.58  frames:  271  epsilon:  0.10
episode:  1161  score:  267.50  avg_score:  192.28  frames:  234  epsilon:  0.10
episode:  1162  score:  237.43  avg_score:  192.15  frames:  281  epsilon:  0.10
episode:  1163  score:  12.68  avg_score:  189.84  frames:  150  epsilon:  0.10
episode:  1164  score:  265.87  avg_score:  190.15  frames:  268  epsilon:  0.10
episode:  1165  score:  269.52  avg_score:  190.54  frames:  266  epsilon:  0.10
episode:  1166  score:  15.27  avg_score:  188.27  frames:  158  epsilon:  0.10
episode:  1167  score:  270.88  avg_score:  188.68  frames:  308  epsilon:  0.10
episode:  1168  score:  20.98  avg_score:  186.46  frames:  163  epsilon:  0.10
episode:  1169  score:  255.86  avg_score:  186.38  frames:  285  epsilon:  0.10
episode:  1170  score:  276.67  avg_score:  186.47  frames:  348  epsilon:  0.10
episode:  1171  score:  255.89  avg_score:  186.19  frames:  240  epsilon:  0.10
episode:  1172  score:  -263.53  avg_score:  180.91  frames:  64  epsilon:  0.10
episode:  1173  score:  9.12  avg_score:  181.28  frames:  95  epsilon:  0.10
episode:  1174  score:  -76.54  avg_score:  178.94  frames:  71  epsilon:  0.10
episode:  1175  score:  -143.98  avg_score:  177.24  frames:  72  epsilon:  0.10
episode:  1176  score:  252.72  avg_score:  179.60  frames:  304  epsilon:  0.10
episode:  1177  score:  266.55  avg_score:  179.80  frames:  370  epsilon:  0.10
episode:  1178  score:  232.28  avg_score:  181.79  frames:  296  epsilon:  0.09
episode:  1179  score:  243.72  avg_score:  181.16  frames:  393  epsilon:  0.09
episode:  1180  score:  271.68  avg_score:  181.11  frames:  325  epsilon:  0.09
episode:  1181  score:  239.60  avg_score:  180.83  frames:  208  epsilon:  0.09
episode:  1182  score:  11.28  avg_score:  181.27  frames:  175  epsilon:  0.09
episode:  1183  score:  281.22  avg_score:  181.62  frames:  221  epsilon:  0.09
episode:  1184  score:  284.09  avg_score:  181.48  frames:  274  epsilon:  0.09
episode:  1185  score:  255.55  avg_score:  181.69  frames:  269  epsilon:  0.09
episode:  1186  score:  293.87  avg_score:  182.37  frames:  304  epsilon:  0.09
episode:  1187  score:  251.63  avg_score:  182.43  frames:  285  epsilon:  0.09
episode:  1188  score:  296.56  avg_score:  184.87  frames:  291  epsilon:  0.09
episode:  1189  score:  292.94  avg_score:  189.04  frames:  330  epsilon:  0.09
episode:  1190  score:  -8.34  avg_score:  186.34  frames:  125  epsilon:  0.09
episode:  1191  score:  -9.74  avg_score:  186.00  frames:  146  epsilon:  0.09
episode:  1192  score:  43.16  avg_score:  184.09  frames:  117  epsilon:  0.09
episode:  1193  score:  57.33  avg_score:  182.12  frames:  182  epsilon:  0.09
episode:  1194  score:  284.69  avg_score:  182.10  frames:  340  epsilon:  0.09
episode:  1195  score:  284.30  avg_score:  184.46  frames:  346  epsilon:  0.09
episode:  1196  score:  269.47  avg_score:  184.63  frames:  293  epsilon:  0.09
episode:  1197  score:  243.59  avg_score:  184.68  frames:  305  epsilon:  0.09
episode:  1198  score:  263.63  avg_score:  184.77  frames:  425  epsilon:  0.09
episode:  1199  score:  42.84  avg_score:  182.54  frames:  999  epsilon:  0.09
episode:  1200  score:  269.43  avg_score:  182.46  frames:  515  epsilon:  0.09
episode:  1201  score:  26.39  avg_score:  185.90  frames:  125  epsilon:  0.09
episode:  1202  score:  229.25  avg_score:  191.45  frames:  310  epsilon:  0.09
episode:  1203  score:  263.50  avg_score:  191.60  frames:  248  epsilon:  0.09
episode:  1204  score:  254.84  avg_score:  192.04  frames:  289  epsilon:  0.09
episode:  1205  score:  -371.86  avg_score:  185.39  frames:  145  epsilon:  0.09
episode:  1206  score:  16.64  avg_score:  186.22  frames:  999  epsilon:  0.09
episode:  1207  score:  32.61  avg_score:  184.36  frames:  116  epsilon:  0.09
episode:  1208  score:  267.62  avg_score:  184.32  frames:  333  epsilon:  0.09
episode:  1209  score:  -29.69  avg_score:  181.36  frames:  139  epsilon:  0.09
episode:  1210  score:  -13.96  avg_score:  178.48  frames:  145  epsilon:  0.09
episode:  1211  score:  -409.28  avg_score:  171.64  frames:  220  epsilon:  0.09
episode:  1212  score:  -165.61  avg_score:  167.09  frames:  220  epsilon:  0.09
episode:  1213  score:  197.65  avg_score:  166.18  frames:  277  epsilon:  0.09
episode:  1214  score:  146.39  avg_score:  164.69  frames:  901  epsilon:  0.09
episode:  1215  score:  229.76  avg_score:  164.29  frames:  560  epsilon:  0.09
episode:  1216  score:  259.79  avg_score:  163.80  frames:  368  epsilon:  0.09
episode:  1217  score:  -351.97  avg_score:  157.74  frames:  135  epsilon:  0.09
episode:  1218  score:  230.21  avg_score:  157.29  frames:  297  epsilon:  0.09
episode:  1219  score:  -367.69  avg_score:  152.03  frames:  260  epsilon:  0.09
episode:  1220  score:  252.92  avg_score:  151.96  frames:  655  epsilon:  0.09
episode:  1221  score:  211.64  avg_score:  151.52  frames:  985  epsilon:  0.09
episode:  1222  score:  13.18  avg_score:  149.54  frames:  115  epsilon:  0.09
episode:  1223  score:  254.74  avg_score:  151.70  frames:  345  epsilon:  0.09
episode:  1224  score:  264.83  avg_score:  154.26  frames:  293  epsilon:  0.09
episode:  1225  score:  137.21  avg_score:  152.91  frames:  999  epsilon:  0.09
episode:  1226  score:  2.14  avg_score:  154.99  frames:  999  epsilon:  0.09
episode:  1227  score:  172.48  avg_score:  154.14  frames:  873  epsilon:  0.09
episode:  1228  score:  238.69  avg_score:  153.91  frames:  448  epsilon:  0.09
episode:  1229  score:  264.28  avg_score:  153.91  frames:  316  epsilon:  0.09
episode:  1230  score:  278.16  avg_score:  154.08  frames:  387  epsilon:  0.09
episode:  1231  score:  268.74  avg_score:  158.09  frames:  319  epsilon:  0.09
episode:  1232  score:  288.04  avg_score:  162.05  frames:  284  epsilon:  0.09
episode:  1233  score:  295.54  avg_score:  162.18  frames:  381  epsilon:  0.09
episode:  1234  score:  245.06  avg_score:  161.58  frames:  300  epsilon:  0.08
episode:  1235  score:  260.62  avg_score:  161.18  frames:  232  epsilon:  0.08
episode:  1236  score:  295.37  avg_score:  161.27  frames:  263  epsilon:  0.08
episode:  1237  score:  315.50  avg_score:  161.75  frames:  380  epsilon:  0.08
episode:  1238  score:  294.70  avg_score:  164.47  frames:  312  epsilon:  0.08
episode:  1239  score:  228.55  avg_score:  164.41  frames:  443  epsilon:  0.08
episode:  1240  score:  280.84  avg_score:  164.63  frames:  258  epsilon:  0.08
episode:  1241  score:  249.34  avg_score:  166.80  frames:  238  epsilon:  0.08
episode:  1242  score:  276.77  avg_score:  166.94  frames:  254  epsilon:  0.08
episode:  1243  score:  230.47  avg_score:  166.72  frames:  236  epsilon:  0.08
episode:  1244  score:  260.39  avg_score:  166.32  frames:  312  epsilon:  0.08
episode:  1245  score:  249.95  avg_score:  166.34  frames:  318  epsilon:  0.08
episode:  1246  score:  265.79  avg_score:  169.00  frames:  252  epsilon:  0.08
episode:  1247  score:  233.66  avg_score:  168.69  frames:  259  epsilon:  0.08
episode:  1248  score:  261.21  avg_score:  168.47  frames:  255  epsilon:  0.08
episode:  1249  score:  253.85  avg_score:  168.40  frames:  229  epsilon:  0.08
episode:  1250  score:  280.72  avg_score:  168.36  frames:  277  epsilon:  0.08
episode:  1251  score:  278.80  avg_score:  171.08  frames:  379  epsilon:  0.08
episode:  1252  score:  298.21  avg_score:  171.70  frames:  236  epsilon:  0.08
episode:  1253  score:  255.76  avg_score:  171.79  frames:  219  epsilon:  0.08
episode:  1254  score:  256.29  avg_score:  171.54  frames:  243  epsilon:  0.08
episode:  1255  score:  270.51  avg_score:  174.40  frames:  267  epsilon:  0.08
episode:  1256  score:  285.10  avg_score:  174.62  frames:  336  epsilon:  0.08
episode:  1257  score:  268.92  avg_score:  174.69  frames:  205  epsilon:  0.08
episode:  1258  score:  275.22  avg_score:  174.78  frames:  181  epsilon:  0.08
episode:  1259  score:  262.38  avg_score:  174.76  frames:  319  epsilon:  0.08
episode:  1260  score:  244.21  avg_score:  174.53  frames:  218  epsilon:  0.08
episode:  1261  score:  271.11  avg_score:  174.57  frames:  240  epsilon:  0.08
episode:  1262  score:  247.02  avg_score:  174.66  frames:  404  epsilon:  0.08
episode:  1263  score:  216.66  avg_score:  176.70  frames:  320  epsilon:  0.08
episode:  1264  score:  286.94  avg_score:  176.91  frames:  288  epsilon:  0.08
episode:  1265  score:  282.25  avg_score:  177.04  frames:  180  epsilon:  0.08
episode:  1266  score:  256.29  avg_score:  179.45  frames:  247  epsilon:  0.08
episode:  1267  score:  281.30  avg_score:  179.55  frames:  320  epsilon:  0.08
episode:  1268  score:  269.76  avg_score:  182.04  frames:  249  epsilon:  0.08
episode:  1269  score:  248.96  avg_score:  181.97  frames:  316  epsilon:  0.08
episode:  1270  score:  268.45  avg_score:  181.89  frames:  303  epsilon:  0.08
episode:  1271  score:  216.47  avg_score:  181.50  frames:  459  epsilon:  0.08
episode:  1272  score:  273.39  avg_score:  186.87  frames:  334  epsilon:  0.08
episode:  1273  score:  275.06  avg_score:  189.53  frames:  259  epsilon:  0.08
episode:  1274  score:  250.31  avg_score:  192.79  frames:  365  epsilon:  0.08
episode:  1275  score:  275.99  avg_score:  196.99  frames:  291  epsilon:  0.08
episode:  1276  score:  257.99  avg_score:  197.05  frames:  280  epsilon:  0.08
episode:  1277  score:  -130.06  avg_score:  193.08  frames:  48  epsilon:  0.08
episode:  1278  score:  231.62  avg_score:  193.07  frames:  506  epsilon:  0.08
episode:  1279  score:  281.62  avg_score:  193.45  frames:  259  epsilon:  0.08
episode:  1280  score:  285.23  avg_score:  193.59  frames:  380  epsilon:  0.08
episode:  1281  score:  277.81  avg_score:  193.97  frames:  499  epsilon:  0.08
episode:  1282  score:  282.65  avg_score:  196.68  frames:  298  epsilon:  0.08
episode:  1283  score:  281.32  avg_score:  196.69  frames:  349  epsilon:  0.08
episode:  1284  score:  234.27  avg_score:  196.19  frames:  272  epsilon:  0.08
episode:  1285  score:  270.24  avg_score:  196.33  frames:  289  epsilon:  0.08
episode:  1286  score:  289.71  avg_score:  196.29  frames:  260  epsilon:  0.08
episode:  1287  score:  -5.36  avg_score:  193.72  frames:  173  epsilon:  0.08
episode:  1288  score:  244.64  avg_score:  193.20  frames:  191  epsilon:  0.08
episode:  1289  score:  250.30  avg_score:  192.78  frames:  384  epsilon:  0.08
episode:  1290  score:  -313.06  avg_score:  189.73  frames:  929  epsilon:  0.08
episode:  1291  score:  252.85  avg_score:  192.36  frames:  295  epsilon:  0.08
episode:  1292  score:  274.53  avg_score:  194.67  frames:  228  epsilon:  0.08
episode:  1293  score:  298.43  avg_score:  197.08  frames:  358  epsilon:  0.08
episode:  1294  score:  247.15  avg_score:  196.70  frames:  440  epsilon:  0.08
episode:  1295  score:  235.14  avg_score:  196.21  frames:  679  epsilon:  0.08
episode:  1296  score:  231.40  avg_score:  195.83  frames:  258  epsilon:  0.07
episode:  1297  score:  276.39  avg_score:  196.16  frames:  328  epsilon:  0.07
episode:  1298  score:  250.64  avg_score:  196.03  frames:  337  epsilon:  0.07
episode:  1299  score:  279.74  avg_score:  198.40  frames:  202  epsilon:  0.07
episode:  1300  score:  180.66  avg_score:  197.51  frames:  584  epsilon:  0.07
episode:  1301  score:  257.82  avg_score:  199.83  frames:  297  epsilon:  0.07
episode:  1302  score:  253.48  avg_score:  200.07  frames:  268  epsilon:  0.07
episode:  1303  score:  215.63  avg_score:  199.59  frames:  266  epsilon:  0.07
episode:  1304  score:  266.00  avg_score:  199.70  frames:  199  epsilon:  0.07
episode:  1305  score:  50.95  avg_score:  203.93  frames:  173  epsilon:  0.07
episode:  1306  score:  263.49  avg_score:  206.40  frames:  204  epsilon:  0.07
episode:  1307  score:  296.53  avg_score:  209.04  frames:  269  epsilon:  0.07
episode:  1308  score:  33.75  avg_score:  206.70  frames:  235  epsilon:  0.07
episode:  1309  score:  16.66  avg_score:  207.16  frames:  999  epsilon:  0.07
episode:  1310  score:  146.34  avg_score:  208.76  frames:  978  epsilon:  0.07
episode:  1311  score:  273.11  avg_score:  215.59  frames:  397  epsilon:  0.07
episode:  1312  score:  265.40  avg_score:  219.90  frames:  284  epsilon:  0.07
episode:  1313  score:  291.51  avg_score:  220.84  frames:  221  epsilon:  0.07
episode:  1314  score:  286.35  avg_score:  222.24  frames:  234  epsilon:  0.07
episode:  1315  score:  -333.71  avg_score:  216.60  frames:  374  epsilon:  0.07
episode:  1316  score:  278.90  avg_score:  216.79  frames:  191  epsilon:  0.07
episode:  1317  score:  253.29  avg_score:  222.85  frames:  201  epsilon:  0.07
episode:  1318  score:  278.30  avg_score:  223.33  frames:  326  epsilon:  0.07
episode:  1319  score:  257.73  avg_score:  229.58  frames:  248  epsilon:  0.07
episode:  1320  score:  287.55  avg_score:  229.93  frames:  345  epsilon:  0.07
episode:  1321  score:  278.31  avg_score:  230.59  frames:  321  epsilon:  0.07
episode:  1322  score:  305.23  avg_score:  233.51  frames:  263  epsilon:  0.07
episode:  1323  score:  -431.09  avg_score:  226.66  frames:  158  epsilon:  0.07
episode:  1324  score:  -5.72  avg_score:  223.95  frames:  999  epsilon:  0.07
episode:  1325  score:  262.13  avg_score:  225.20  frames:  270  epsilon:  0.07
episode:  1326  score:  270.38  avg_score:  227.88  frames:  318  epsilon:  0.07
episode:  1327  score:  262.60  avg_score:  228.78  frames:  333  epsilon:  0.07
episode:  1328  score:  273.13  avg_score:  229.13  frames:  240  epsilon:  0.07
episode:  1329  score:  238.69  avg_score:  228.87  frames:  239  epsilon:  0.07
episode:  1330  score:  275.40  avg_score:  228.84  frames:  327  epsilon:  0.07
episode:  1331  score:  260.00  avg_score:  228.76  frames:  300  epsilon:  0.07
episode:  1332  score:  -3.71  avg_score:  225.84  frames:  110  epsilon:  0.07
episode:  1333  score:  -10.36  avg_score:  222.78  frames:  124  epsilon:  0.07
episode:  1334  score:  264.21  avg_score:  222.97  frames:  188  epsilon:  0.07
episode:  1335  score:  282.59  avg_score:  223.19  frames:  352  epsilon:  0.07
episode:  1336  score:  220.22  avg_score:  222.44  frames:  326  epsilon:  0.07
episode:  1337  score:  263.09  avg_score:  221.92  frames:  288  epsilon:  0.07
episode:  1338  score:  248.04  avg_score:  221.45  frames:  258  epsilon:  0.07
episode:  1339  score:  242.73  avg_score:  221.59  frames:  249  epsilon:  0.07
episode:  1340  score:  229.64  avg_score:  221.08  frames:  340  epsilon:  0.07
episode:  1341  score:  285.29  avg_score:  221.44  frames:  269  epsilon:  0.07
episode:  1342  score:  86.80  avg_score:  219.54  frames:  875  epsilon:  0.07
episode:  1343  score:  220.32  avg_score:  219.44  frames:  318  epsilon:  0.07
"""

In [47]:


with open('weights_DQN_1-0.0001-0.002-0.99-30-0.0005.json', 'wb') as fp:
    pickle.dump({'rewards':[float(i.split(" ")[6]) for i in string.split('\n') if i!='']}, fp)
